# 뉴진스, 블랙핑크, 르세라핌 News+Blog키워드 비교






- '르세라핌' 파트는 뉴스 크롤링 완료 되면 추가 예정

In [142]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [146]:
import pandas as pd
import numpy as np
import torch
from tqdm.auto import tqdm
import random
import os

def reset_seeds(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True

DATA_PATH = "/content/drive/MyDrive/멀티캠퍼스 자료/Machine Learning/data/"
SEED = 42

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device

'cuda'

## 크롤링 결과를 하나의 텍스트화 (title + content)

- 뉴진스

In [3]:
train_1 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_05.csv")
train_2 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_06.csv")
train_3 = pd.read_csv(f"{DATA_PATH}뉴진스_뉴스_07.csv")
train_4 = pd.read_csv(f"{DATA_PATH}뉴진스_블로그_3개월.csv", encoding='cp949')

In [4]:
train_1.shape , train_2.shape , train_3.shape , train_4.shape

((1030, 4), (912, 4), (595, 4), (148, 4))

In [5]:
train_nj = pd.concat([train_1,train_2,train_3,train_4],axis=0)
train_nj.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오전 10:06,\n\t\t\t'그래미 5관왕' 존 바티스트 신곡에 뉴진스 협업\n\t\t,https://n.news.naver.com/mnews/article/003/001...,[\n'코크 스튜디오' 위해 만든 싱글 '비 후 유 아' 작업 \n[서울=뉴시스] ...,NaN,NaN
1,2023.05.31. 오후 3:31,"\n\t\t\t""'품위녀'와 전혀 달라"" K-맘 뉴진스의 '행복배틀'[종합]\n\t\t",https://n.news.naver.com/mnews/article/437/000...,[\n \n31일 오후 서울 마포구 상암동 스탠포드호텔에서 ENA '행복배틀'...,NaN,NaN
2,2023.05.31. 오후 3:40,"\n\t\t\t뉴진스, 블랙핑크 그리고 ‘행복배틀’…이엘·진서연→우정원, 케이맘이 ...",https://n.news.naver.com/mnews/article/057/000...,"[\n \n‘행복배틀’ 제작발표회 사진=김영구 MK스포츠 기자‘행복배틀’ 이엘, 진...",NaN,NaN
3,2023.05.31. 오후 3:16,"\n\t\t\t""뉴진스·블랙핑크 이전에 K-맘"" 진서연→이엘 '행복배틀' 발발[종합...",https://n.news.naver.com/mnews/article/108/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[스타뉴스 | 상암(서울)=...,NaN,NaN
4,2023.05.31. 오후 6:25,\n\t\t\t뉴진스·아이브·에스파 이어 (여자)아이들…'퀸카' 멜론 일간차트 1위...,https://n.news.naver.com/mnews/article/003/001...,"[\n미니 6집 '아이 필', '빌보드 200' 41위 자체 최고 기록 \n[서울=...",NaN,NaN


In [6]:
train_nj['title'] = train_nj['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_nj['content'] = train_nj['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
nj_text = train_nj['title'] + train_nj['content']
nj_text

0      그래미 5관왕 존 바티스트 신곡에 뉴진스 협업코크 스튜디오 위해 만든 싱글 비 후 ...
1      품위녀와 전혀 달라 K맘 뉴진스의 행복배틀종합    31일 오후 서울 마포구 상암동...
2      뉴진스, 블랙핑크 그리고 행복배틀이엘진서연우정원, 케이맘이 온다종합M이슈 행복배틀 ...
3      뉴진스블랙핑크 이전에 K맘 진서연이엘 행복배틀 발발종합스타뉴스  상암서울김노을 기자...
4      뉴진스아이브에스파 이어 여자아이들퀸카 멜론 일간차트 1위미니 6집 아이 필, 빌보드...
                             ...                        
143    뉴진스 토끼티  포토카드  앨범기밀이라고 해서, 거참 어디가서 말할 수는 없지만,뉴...
144    오렌즈 프렌츠샤인 원데이 웜톤  쿨톤을 위한 뉴진스 렌즈오렌즈에서 프렌츠샤인이라는신...
145    뉴진스 파워퍼프걸 캐릭터 만들기 테스트 링크 mbti 의외로 ENFJ미국의 키치한 ...
146    뉴진스의 사인 축구 셔츠를 받을 수 있는 이벤트 공개  with 카포스토어대한민국의...
147    뉴진스 파워퍼프걸 콜라보 케이스티파이 powerpuff yourself파워퍼프걸 뉴...
Length: 2685, dtype: object

- 블랙핑크

In [7]:
train_1 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_5월.csv")
train_2 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_6월.csv")
train_3 = pd.read_csv(f"{DATA_PATH}블랙핑크_뉴스_7월.csv")
train_4 = pd.read_csv(f"{DATA_PATH}블랙핑크_블로그_3개월.csv", encoding='utf-8')

In [8]:
train_1.shape , train_2.shape , train_3.shape , train_4.shape

((1125, 4), (387, 4), (235, 4), (180, 4))

In [9]:
train_bp = pd.concat([train_1,train_2,train_3,train_4],axis=0)
train_bp.head()

,date,title,link,content,Unnamed: 0,datetime
0,2023.05.31. 오후 9:17,\n\t\t\t블랙핑크와 게임의 만남\n\t\t,https://n.news.naver.com/mnews/article/469/000...,"[\n'블랙핑크 더 게임', 지난 18일 글로벌 출시""블랙핑크 매력 보여줄 수 있는...",NaN,NaN
1,2023-05-31 20:00:01,[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?,https://n.news.naver.com/mnews/article/001/001...,[\n\t\t\t[쉬운 우리말 쓰기] 블랙핑크가 맡은 '헤드라이너'가 뭔가요?[ ※...,NaN,NaN
2,2023-05-30 08:29:03,"유안타證 ""와이지엔터, 블랙핑크 성장세 지속""",https://n.news.naver.com/mnews/article/003/001...,[\n\n\n\n\n(자료제공=유안타증권) *재판매 및 DB 금지[서울=뉴시스]이지...,NaN,NaN
3,2023.05.30. 오전 9:05,"\n\t\t\t'블랙핑크 여동생' 베이비 몬스터, 심상치 않은 인기..글로벌팬 공감...",https://n.news.naver.com/mnews/article/112/000...,[\n\t\t\t\t\n\t\t\t\t\n\t\t\t\t[헤럴드POP=김지혜 기자]...,NaN,NaN
4,2023.05.31. 오후 3:11,"\n\t\t\t""블랙핑크·뉴진스 전에 K맘""…'행복배틀' 이엘→진서연의 스릴러 [종...",https://n.news.naver.com/mnews/article/311/000...,[\n \n(엑스포츠뉴스 김현정 기자) ‘행복배틀’이 치열한 심리전과 예측불가 전개...,NaN,NaN


In [10]:
train_bp['title'] = train_bp['title'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)
train_bp['content'] = train_bp['content'].str.replace("[^a-zA-Z가-힣ㄱ-ㅎ0-9.,!? ]" , "",regex=True)

# 제목, 본문내용 합치기
bp_text = train_bp['title'] + train_bp['content']
bp_text

0      블랙핑크와 게임의 만남블랙핑크 더 게임, 지난 18일 글로벌 출시블랙핑크 매력 보여...
1      쉬운 우리말 쓰기 블랙핑크가 맡은 헤드라이너가 뭔가요?쉬운 우리말 쓰기 블랙핑크가 ...
2      유안타 와이지엔터, 블랙핑크 성장세 지속자료제공유안타증권 재판매 및 DB 금지서울뉴...
3      블랙핑크 여동생 베이비 몬스터, 심상치 않은 인기..글로벌팬 공감대헤럴드POP김지혜...
4      블랙핑크뉴진스 전에 K맘행복배틀 이엘진서연의 스릴러 종합 엑스포츠뉴스 김현정 기자 ...
                             ...                        
175    스타벅스 신메뉴 블랙핑크 콜라보 제품 후기 디저트 먹고 꿀팁스벅앱을 들어가니 뜬 소...
176    블랙핑크 제니 열애설 상기 시그널? 방탄소년단BTS 뷔와 파리 데이트룩 자발적 공개...
177    연예인향수 궁금하지 미스디올 블랙핑크 지수 강민경 엄정화 손석구 향수안녕하세요 다양...
178    스벅신상 블랙핑크 딸기 크림 초코번 맛있네제가 좋아하는 두 조합!!블랙핑크  스타벅...
179    베트남 하노이 블랙핑크 환영2층버스 팬들로 가득베트남 하노이 블랙핑크 환영2층버스 ...
Length: 1927, dtype: object

In [11]:
len(nj_text), len(bp_text)

(2685, 1927)

## Kiwi로 토큰화 및 형용사 추출

In [12]:
!pip install kiwipiepy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 30.5/30.5 MB 53.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for kiwipiepy-model: filename=kiwipiepy_model-0.15.0-py3-none-any.whl size=30602629 sha256=b29f1db29bb92dbc3bd1ba39c01d344448c2a847736d84245d717ebf3c9770db
  Stored in directory: /root/.cache/pip/wheels/f3/55/41/ca474338ece1bc4314b01445f64ff002d71e19df45575a16e2
Successfully built kiwipiepy-model


In [13]:
from kiwipiepy import Kiwi
from kiwipiepy.utils import Stopwords
stopwords =  Stopwords()
kiwi = Kiwi()

In [14]:
nj_va_keyword = []

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_keyword.append(word)


In [15]:
nj_va_keyword

['폭넓',
 '있',
 '있',
 '다르',
 '느리',
 '예쁘',
 '다르',
 '많',
 '있',
 '재미있',
 '그렇',
 '많',
 '있',
 '세',
 '좋',
 '좋',
 '많',
 '크',
 '짧',
 '예쁘',
 '높',
 '다르',
 '예쁘',
 '그렇',
 '그렇',
 '재밌',
 '그렇',
 '많',
 '그렇',
 '뻔뻔하',
 '비슷하',
 '많',
 '그렇',
 '좋',
 '크',
 '기쁘',
 '크',
 '많',
 '재밌',
 '많',
 '강하',
 '다르',
 '좋',
 '예쁘',
 '깊',
 '재밌',
 '많',
 '있',
 '궁금하',
 '크',
 '많',
 '있',
 '크',
 '있',
 '있',
 '있',
 '그렇',
 '많',
 '젊',
 '재밌',
 '있',
 '재미있',
 '있',
 '느리',
 '크',
 '크',
 '크',
 '좋',
 '뻔뻔하',
 '재미있',
 '못지않',
 '다르',
 '좋',
 '있',
 '크',
 '크',
 '있',
 '있',
 '어리',
 '예쁘',
 '크',
 '크',
 '짧',
 '짧',
 '기쁘',
 '그렇',
 '다르',
 '있',
 '재미있',
 '있',
 '저렇',
 '좋',
 '어떻',
 '크',
 '재미있',
 '흥미진진하',
 '힘들',
 '좋',
 '있',
 '있',
 '있',
 '못지않',
 '있',
 '많',
 '있',
 '재밌',
 '뻔하',
 '느리',
 '크',
 '재밌',
 '철없',
 '많',
 '크',
 '어떻',
 '많',
 '재밌',
 '많',
 '그렇',
 '좋',
 '재밌',
 '다르',
 '있',
 '예쁘',
 '깊',
 '재밌',
 '많',
 '멋지',
 '예쁘',
 '높',
 '깊',
 '많',
 '그렇',
 '있',
 '크',
 '강렬하',
 '궁금하',
 '크',
 '많',
 '있',
 '다르',
 '어떻',
 '재밌',
 '어리',
 '좋',
 '길',
 '짧',
 '있',
 '기쁘',
 '길',
 '크',
 '기쁘',
 '있',
 '맛있

- 블랙핑크


In [16]:
bp_text = [text for text in bp_text if isinstance(text, str)]
#float 값 때문에 오류나는 것 제거

In [17]:
bp_va_keyword = []

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사
    for idx, word in enumerate(tmp) :
        bp_va_keyword.append(word)

- 르세라핌

In [18]:
# ls_va_keyword = []

# for idx, text in enumerate(ls_text) :
#     result = kiwi.tokenize(text,stopwords=stopwords)
#     tmp = [token.form for token in result if token.tag in ['VA']] #형용사
#     for idx, word in enumerate(tmp) :
#         ls_va_keyword.append(word)

- 형용사 개수 (1500~2500개)

In [20]:
len(nj_va_keyword), len(bp_va_keyword),
#len(ls_va_keyword)

(17146, 6368)

## 불용어 및 동의어 처리
- 예쁨-이쁨, 재미있음-재밌음

- 불용어 처리

In [ ]:
stopword_list = ['어떻','이렇','그렇','저렇','요렇','있','이러하','저러하', #의미없음
                 '많','적','길','수많','커다랗','각지','자세하','같','비슷하', #의미 있으나 사람 표현 아님
                 '답답하','귀찮','힘들', #의미 있으나 인물에 대한 반응 아님
                 '좋','싫','이쁘','재미있'] #중복 (이쁘-예쁘, 재미있-재밌) or 너무 넓은 의미 (좋, 싫)

nj_nva_list = [x for x in nj_va_keyword if x not in stopword_list]
bp_nva_list = [x for x in bp_va_keyword if x not in stopword_list]
ls_nva_list = [x for x in ls_va_keyword if x not in stopword_list]

## 형용사를 명사화

- 형용사를 명사화하는 함수 적용 (adjective_to_noun)
    - 마지막 글자에 받침이 있으면 '음'을, 없으면 받침에 ㅁ을 더함
    

In [ ]:
def adjective_to_noun(adjective_list):
    results = []
    for adjective in adjective_list:
        last_char = adjective[-1]

        if (ord(last_char) - 44032) % 28 != 0: #받침이 있다면 음을 더하고
            result = adjective + "음"
        else:
            last_jamo = chr(ord(last_char) + 16) #없다면 마지막 글자에 ㅁ 받침을 추가
            result = adjective[:-1] + last_jamo

        results.append(result)
    return results

In [ ]:
nj_nva_list = adjective_to_noun(nj_nva_list)
bp_nva_list = adjective_to_noun(bp_nva_list)
ls_nva_list = adjective_to_noun(ls_nva_list)

In [ ]:
ls_nva_list

## TF-IDF 기반 top 20 키워드 선정

- TF-IDF 함수 정의

In [40]:
!pip install konlpy

In [39]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
def extract_keywords_tfidf(text, num_keywords=5):
    kkma = Kkma()
    # nouns = kkma.nouns(text)
    # preprocessed_text = " ".join(nouns)

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # TF-IDF 값에 따라 정렬된 단어들의 인덱스를 얻음
    sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]

   # 상위 num_keywords 개의 단어와 해당 단어들의 TF-IDF 스코어 추출
    top_keywords = [(feature_names[idx], tfidf_matrix[0, idx]) for idx in sorted_indices[:num_keywords]]

    return top_keywords

- 세 텍스트에 대해 TF-IDF 키워드 추출

In [ ]:
nj_nva_string = ', '.join(nj_nva_list)
bp_nva_string = ', '.join(bp_nva_list)
ls_nva_string = ', '.join(ls_nva_list)

- 뉴진스

In [ ]:
text = nj_nva_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

nj_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

- 블랙핑크

In [ ]:
text = bp_nva_string

top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

bp_tfidf_top20 = []

print("TF-IDF 기반 키워드 및 스코어 추출:")
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top20.append(keyword)
    print(f"{keyword} - Score: {score}")

- 르세라핌

In [ ]:
# text = ls_nva_string

# top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=20)

# ls_tfidf_top20 = []

# print("TF-IDF 기반 키워드 및 스코어 추출:")
# for keyword, score in top_keywords_tfidf:
#     ls_tfidf_top20.append(keyword)
#     print(f"{keyword} - Score: {score}")

In [ ]:
df_nj = pd.DataFrame(nj_tfidf_top20)
df_bp = pd.DataFrame(bp_tfidf_top20)
# df_ls = pd.DataFrame(ls_tfidf_top20)

df_nj.columns = ['뉴진스']
df_bp.columns = ['블랙핑크']
# df_ls.columns = ['르세라핌']

pd.concat([df_nj, df_bp],axis=1)

## 키워드 분석 및 비교

### 공통 및 개별 키워드

- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top20




In [ ]:
# 각 리스트를 집합(set)으로 변환
nj_set_top20 = set(nj_tfidf_top20)
bp_set_top20 = set(bp_tfidf_top20)
# ls_set_top20 = set(ls_tfidf_top20)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top20.intersection(bp_set_top20)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords))

In [ ]:
# common list에 있는 키워드 제외
nj_unique_top20 = nj_set_top20.difference(common_keywords)
bp_unique_top20 = bp_set_top20.difference(common_keywords)
# ls_unique_top20 = ls_set_top20.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top20)
print("블랙핑크 키워드:", bp_unique_top20)
# print("르세라핌 키워드:", ls_unique_top20)

In [ ]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top20 = nj_set_top20.difference(bp_set_top20)
bp_unique_top20 = bp_set_top20.difference(nj_set_top20)
# ls_unique_top20 = ls_set_top20.difference(nj_set_top20, bp_set_top20)

print("뉴진스 키워드:", nj_unique_top20)
print("블랙핑크 키워드:", bp_unique_top20)
# print("르세라핌 키워드:", ls_unique_top20)

- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top50


In [ ]:
text = nj_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

nj_tfidf_top50 = []
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top50.append(keyword)

In [ ]:
text = bp_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

bp_tfidf_top50 = []
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top50.append(keyword)

In [ ]:
# text = ls_nva_string
# top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=50)

# ls_tfidf_top50 = []
# for keyword, score in top_keywords_tfidf:
#     ls_tfidf_top50.append(keyword)

In [ ]:
# 각 리스트를 집합(set)으로 변환
nj_set_top50 = set(nj_tfidf_top50)
bp_set_top50 = set(bp_tfidf_top50)
# ls_set_top50 = set(ls_tfidf_top50)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top50.intersection(bp_set_top50)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords))

In [ ]:
# common list에 있는 키워드 제외
nj_unique_top50 = nj_set_top50.difference(common_keywords)
bp_unique_top50 = bp_set_top50.difference(common_keywords)
# ls_unique_top50 = ls_set_top50.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top50)
print("블랙핑크 키워드:", bp_unique_top50)
# print("르세라핌 키워드:", ls_unique_top50)

In [ ]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top50 = nj_set_top50.difference(bp_set_top50)
bp_unique_top50 = bp_set_top50.difference(nj_set_top50)
# ls_unique_top50 = ls_set_top50.difference(nj_set_top50, bp_set_top50)

print("뉴진스 키워드:", nj_unique_top50)
print("블랙핑크 키워드:", bp_unique_top50)
# print("르세라핌 키워드:", ls_unique_top50)

- 뉴진스, 블랙핑크, 르세라핌 공통 키워드 : Top100


In [ ]:
text = nj_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

nj_tfidf_top100 = []
for keyword, score in top_keywords_tfidf:
    nj_tfidf_top100.append(keyword)

In [ ]:
text = bp_nva_string
top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

bp_tfidf_top100 = []
for keyword, score in top_keywords_tfidf:
    bp_tfidf_top100.append(keyword)

In [ ]:
# text = ls_nva_string
# top_keywords_tfidf = extract_keywords_tfidf(text,num_keywords=100)

# ls_tfidf_top100 = []
# for keyword, score in top_keywords_tfidf:
#     ls_tfidf_top100.append(keyword)

In [ ]:
# 각 리스트를 집합(set)으로 변환
nj_set_top100 = set(nj_tfidf_top100)
bp_set_top100 = set(bp_tfidf_top100)
# ls_set_top100 = set(ls_tfidf_top100)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_set_top100.intersection(bp_set_top100)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords))

In [ ]:
# common list에 있는 키워드 제외
nj_unique_top100 = nj_set_top100.difference(common_keywords)
bp_unique_top100 = bp_set_top100.difference(common_keywords)
# ls_unique_top100 = ls_set_top100.difference(common_keywords)

print("뉴진스 키워드:", nj_unique_top100)
print("블랙핑크 키워드:", bp_unique_top100)
# print("르세라핌 키워드:", ls_unique_top100)

In [ ]:
# 각자 서로 다른 키워드: 각 리스트에만 있는 키워드
nj_unique_top100 = nj_set_top50.difference(bp_set_top100)
bp_unique_top100 = bp_set_top50.difference(nj_set_top100)
# ls_unique_top100 = ls_set_top50.difference(nj_set_top100, bp_set_top100)

print("뉴진스 키워드:", nj_unique_top100)
print("블랙핑크 키워드:", bp_unique_top100)
# print("르세라핌 키워드:", ls_unique_top100)

## Meaning word 작업


In [135]:
va_meaning_words = ['많', '크', '좋','새롭','다르','어렵','쉽','작','길','아름답','어리','힘들','젊','아프','빠르','나쁘','싫','무섭','짧',
                 '커다랗','엄청나','지나치','희','검','어둡','옳','괜찮','밝','바쁘','가볍','뜨겁','하얗','예쁘','무겁','맑','즐겁','부드럽',
                 '재미있','약하','낯설','편하','낫','비싸','강하','슬프','춥','놀랍','곱','뛰어나','낡','거칠','안타깝','부끄럽','기쁘','바르',
                 '두렵','반갑','맛있','흔하','착하','차갑','싸','조그맣','외롭','올바르','빨갛','멋지','날카롭','급하','친하','굵','굳','아쉽',
                 '느리','환하','더럽','그립','괴롭','귀엽','우습','가늘','어리석','귀하','둥글','부럽','노랗','귀찮','재밌','세','까맣','두껍',
                 '어지럽','거세','차','흐리','파랗','밉','아깝','시끄럽','눈부시','얇','진하','힘겹','점잖','고프','까다롭','힘차','폭넓',
                 '못하','잘나','사납','두드러지','끈질기','험하', '이쁘',' 누렇',' 시커멓','오래','낯익','배고프','끊임없','남다르','지겹',
                 '비','쓸데없','못나','벅차','묘하','기막히','서럽','서글프','세차','값싸','그르','모질','가엾','잘','참되','허옇','엷',
                 '색다르','동그랗','게으르','메마르','벌겋','쑥스럽','엄하','어이없','가느다랗','너그럽','서투르','하찮','시리','매끄럽',
                 '우스꽝스럽','고달프','서툴','뻔하','값지','멋있','잘생기','질기','멍하','새까맣','연하','추하','틀리','억세','안되','여리',
                 '싱겁','쓰라리','어처구니없','역겹','재미없','수줍','뿌옇','천하','징그럽','순하','흉하','안쓰럽','야하','빼어나','속상하',
                 '유별나','매섭','보잘것없','못생기','못되','시퍼렇','기다랗','짓궂','언짢','잽싸','형편없','값비싸','흥겹','새파랗','둥그렇',
                 '둔하','부시','재빠르','딱하','배부르','싱그럽','별나','쓰리','야무지','새하얗','어질','한결같','활기차','궂','아리','갸날프',
                 '건방지','독하','선하','훤하','시뻘겋','탁하','무디','우렁차','능하','철없','뽀얗','번거롭','해롭','쓸모없','성가시','정겹',
                 '애처롭','퍼렇','분하','관계없','네모나','얄밉','차디차','장하','이롭','달갑','헤프','앳되','후하','푸르르','쎄','힘세',
                 '힘없','보드랍','새빨갛','길다랗','아니꼽','미심쩍','해맑','미덥','수상쩍','굳세','만만찮','동떨어지','껄끄럽','험상궂',
                 '변함없','드세','앙증맞','깡마르','구슬프','날쌔','실없','쓰디쓰','시꺼멓','을씨년스럽','검푸르','발갛','시원찮','뻘겋',
                 '어여쁘','당차','고되','모나','다부지','흐드러지','드럽','짙푸르','약삭빠르','방정맞','뜸하','사이좋','천연덕스럽','무르',
                 '볼품없','상스럽','새카맣','격하','거침없','허하','희뿌옇','앙칼지','심술궂','아리땁','끈덕지','역하','가녀리','노오랗',
                 '굼뜨','희디희','가엽','살갑','하잘것없','정신없','버릇없','올곧','악하','청승맞','별스럽','구리','샛노랗','길하','빈틈없',
                 '필요없','남부럽','멋들어지','쪼그맣','똥그랗','매몰차','파아랗','속좁','막되','노하','얌전하','얇디얇','얍삽하','연푸르',
                 '번거럽','똘똘하','야물딱지','예쁘장하','투박스럽','게을러빠지','껄렁껄렁하','믿음직스럽','사근사근하','기','될성부르',
                 '약아빠지','예스럽','쇼킹하','놀라웁','맑디맑','능청맞','곱디곱','징하','모지','꽁하','따스','늙','유머러스하','예쁘장스럽',
                 '촌시럽','보오얗','동그렇','모자르','모잘르','간지','힘쎄','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','밉살맞','후덥',
                 '여물','찐하','찡하','둥굴','기차','자애롭','시덥잖','쌍스럽','쑥쓰럽','앙징맞','얄따랗','밉살맞','버얼겋','높푸르','찐하',
                 '찡하','기차','주제넘','꺼멓','어리숙하','얄궂','참하','끄떡없','희망차','기운차','희부옇','안스럽','어줍잖','의롭','능글맞',
                 '간드러지','쌀쌀맞','자그맣','볼썽사납','희멀겋','도드라지','재수없','걸','빠알갛','뜬금없','살지','악착같','멀겋','의좋','꾸밈없',
                 '같잖','잡스럽','냉하','싸하','빡세','궁상맞','밉살스럽','하이얗','익살맞','쬐그맣','암팡지','어쭙잖','분별없','달갑잖','똑바르',
                 '실하','박하','고깝','특별나','새삼스럽','스스럼없','좀스럽','손색없','후지','짠하','시답잖','허물없','변변찮','따습','맹하',
                 '애닯','주책없','뚱하','의심쩍','쬐끄맣','섹시하','곱다랗','시건방지','특출나','옹골차','막돼먹','곰살궂','가소롭','유하','속없',
                 '지각없','거멓','대단찮','기똥차','깊푸르','따시','뜨시','맥없','싯누렇','로맨틱하','도탑','허여멀겋','모지락스럽','향그럽',
                 '옹골지','성마르','성스럽','꼴사납','시답','동글','마르','대차','조그만하','인상깊','야멸차','염치없','쌩뚱맞','매스껍','극성맞',
                 '깊디깊','감명깊','반하','궁하','덜떨어지','나즈막하','요상하','작디작','째쨰하','촌스럽','뽀오얗','색스럽','다사롭','둔덕지',
                 '맵싸하','까아맣','감미롭','짖궂','시덥','어줍','몽글','값있','눈꼴사납']

In [ ]:
def adjective_to_noun(adjective_list):
    results = []
    for adjective in adjective_list:
        last_char = adjective[-1]

        if (ord(last_char) - 44032) % 28 != 0: #받침이 있다면 음을 더하고
            result = adjective + "음"
        else:
            last_jamo = chr(ord(last_char) + 16) #없다면 마지막 글자에 ㅁ 받침을 추가
            result = adjective[:-1] + last_jamo

        results.append(result)
    return results

In [165]:
def xr_to_noun(xr_list):
    results = []
    for xr in xr_list:
        result = xr + '함'

        results.append(result)
    return results

In [166]:
xr_to_noun(xr_meaningwords)

['강력함',
 '훌륭함',
 '바람직함',
 '따뜻함',
 '깨끗함',
 '화려함',
 '궁금함',
 '진정함',
 '진지함',
 '대단함',
 '익숙함',
 '다양함',
 '조용함',
 '시원함',
 '답답함',
 '소중함',
 '활발함',
 '엉뚱함',
 '평범함',
 '엄격함',
 '솔직함',
 '불쌍함',
 '어색함',
 '억울함',
 '절실함',
 '쓸쓸함',
 '생생함',
 '민감함',
 '신선함',
 '강렬함',
 '만만함',
 '끔찍함',
 '지루함',
 '소박함',
 '이상함',
 '단단함',
 '격렬함',
 '한심함',
 '초라함',
 '과감함',
 '튼튼함',
 '유명함',
 '유연함',
 '섬세함',
 '똑똑함',
 '단호함',
 '지독함',
 '섭섭함',
 '차분함',
 '간절함',
 '순진함',
 '애매함',
 '심심함',
 '불쾌함',
 '절박함',
 '은밀함',
 '예민함',
 '깔끔함',
 '싱싱함',
 '달콤함',
 '잔잔함',
 '유쾌함',
 '서늘함',
 '따스함',
 '싸늘함',
 '집요함',
 '성급함',
 '황당함',
 '떳떳함',
 '유치함',
 '지저분함',
 '우아함',
 '과격함',
 '거룩함',
 '위태함',
 '거대함',
 '단정함',
 '느긋함',
 '용감함',
 '저렴함',
 '허전함',
 '든든함',
 '창백함',
 '멍청함',
 '독특함',
 '조급함',
 '급급함',
 '고귀함',
 '생소함',
 '희한함',
 '친근함',
 '어수선함',
 '철저함',
 '거북함',
 '썰렁함',
 '담담함',
 '세심함',
 '경쾌함',
 '허술함',
 '원만함',
 '예리함',
 '야릇함',
 '비겁함',
 '근사함',
 '상쾌함',
 '허름함',
 '진부함',
 '씩씩함',
 '반듯함',
 '비장함',
 '포근함',
 '고약함',
 '고상함',
 '미숙함',
 '정중함',
 '웅장함',
 '아찔함',
 '난해함',
 '안이함',
 '냉혹함',
 '못마땅함',
 '열렬함',
 '고단함',
 '수상함',
 '탄탄

In [ ]:
!pip install konlpy

In [ ]:
from konlpy.tag import Kkma
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

In [ ]:
def extract_keywords_tfidf(text, num_keywords=5):
    kkma = Kkma()
    # nouns = kkma.nouns(text)
    # preprocessed_text = " ".join(nouns)

    tfidf_vectorizer = TfidfVectorizer()
    tfidf_matrix = tfidf_vectorizer.fit_transform([text])
    feature_names = tfidf_vectorizer.get_feature_names_out()

    # TF-IDF 값에 따라 정렬된 단어들의 인덱스를 얻음
    sorted_indices = tfidf_matrix.toarray()[0].argsort()[::-1]

   # 상위 num_keywords 개의 단어와 해당 단어들의 TF-IDF 스코어 추출
    top_keywords = [(feature_names[idx], tfidf_matrix[0, idx]) for idx in sorted_indices[:num_keywords]]

    return top_keywords

### VA - 뉴진스 전체 vs meaningword 키워드 비교

In [273]:
nj_va_keyword = []

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_keyword.append(word)


In [274]:
len(nj_va_keyword)

17146

In [275]:
nj_va_meaningwords =[]

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA'] and token.form in meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        nj_va_meaningwords.append(word)

In [276]:
len(nj_va_meaningwords)

8464

In [277]:
#3월치 뉴진스 블로그 형용사 리스트

len(set(nj_va_keyword)), len(set(nj_va_meaningwords)) #약 100개가 걸러짐

(449, 165)

In [280]:
# 각 리스트를 집합(set)으로 변환
nj_va_set_keyword = set(nj_va_keyword)
nj_va_set_meaningwords = set(nj_va_meaningwords)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_va_set_keyword.intersection(nj_va_set_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'분하', '어리', '거세', '한결같', '드세', '빼어나', '길', '힘차', '까맣', '멋들어지', '부시', '새빨갛', '연하', '냉하', '편하', '뿌옇', '섹시하', '환하', '조그맣', '진하', '얇', '크', '차', '짧', '슬프', '둥글', '재미있', '별나', '당차', '해맑', '눈부시', '뜸하', '빨갛', '순하', '어질', '변함없', '앙증맞', '독하', '후하', '여리', '멍하', '가늘', '아리', '싸', '틀리', '탁하', '많', '강하', '싸하', '잘생기', '힘들', '올바르', '정신없', '어리석', '짓궂', '예쁘', '세차', '우렁차', '묘하', '속상하', '간드러지', '장하', '서투르', '친하', '멋있', '벅차', '값비싸', '엄청나', '동그랗', '낯설', '새하얗', '굵', '잘', '반하', '급하', '잘나', '도드라지', '바쁘', '젊', '밝', '뛰어나', '나쁘', '앳되', '멋지', '뽀얗', '격하', '옹골차', '검', '파랗', '푸르르', '기운차', '굳세', '약하', '값지', '다르', '쓸데없', '능글맞', '새까맣', '비싸', '괜찮', '희', '뻔하', '끊임없', '귀하', '궂', '맛있', '희망차', '짙푸르', '서글프', '짠하', '하얗', '철없', '노랗', '재밌', '낡', '활기차', '커다랗', '어이없', '힘없', '꾸밈없', '쎄', '서툴', '모질', '네모나', '재미없', '빠르', '아프', '둔하', '세', '좋', '기쁘', '역하', '쓰라리', '폭넓', '오래', '기막히', '이쁘', '거침없', '만만찮', '작', '싫', '지나치', '귀찮', '야하', '흔하', '낯익', '질기', '두드러지', '남다르', '색다르', '바르', '고프', '방정맞', '맑', '못생기', '거칠', '익살맞', '거멓', '흐리', '마르

In [282]:
unique_nj_va_keyword = nj_va_set_keyword.difference(common_keywords)
unique_nj_va_meaningwords = nj_va_set_meaningwords.difference(common_keywords)

print("기사 키워드 리스트 :", unique_nj_va_keyword)
print("형태소 키워드 리스트 :", unique_nj_va_meaningwords)


기사 키워드 리스트 : {'도도하', '탄탄하', '수없', '풋풋하', '조급하', '드물', '바람직하', '저렴하', '칼칼하', '복잡하', '자잘하', '틀림없', '화기애애하', '매콤하', '부질없', '어색하', '상큼하', '생생하', '싸늘하', '세심하', '알싸하', '익숙하', '우아하', '난해하', '잦', '맛나', '기발하', '화창하', '친근하', '습하', '빡빡하', '근사하', '무색하', '끈끈하', '짜릿하', '과하', '어리둥절하', '열악하', '괜하', '생뚱맞', '상관없', '별다르', '편찮', '신선하', '조그마하', '비장하', '저렇', '묽', '잠잠하', '수많', '따끈하', '어마어마하', '헛되', '멋쩍', '아득하', '솔깃하', '향긋하', '심심하', '세세하', '찰지', '정교하', '단호하', '시큰둥하', '느슨하', '달콤하', '고소하', '핫하', '답답하', '좋아하', '솔직하', '빵빵하', '뚜렷하', '미미하', '화사하', '아슬하', '부르', '뜻깊', '웅장하', '단단하', '소소하', '지난하', '읎', '비릿하', '열띠', '확실하', '걸맞', '생소하', '난감하', '가파르', '영락없', '대단하', '깊', '위대하', '충분하', '구수하', '밤늦', '깜찍하', '이러하', '뻑뻑하', '쿨하', '섬세하', '난처하', '자그마하', '불가피하', '더디', '즐비하', '설', '구성지', '평범하', '달', '어눌하', '서운하', '씁쓸하', '간략하', '무난하', '느끼하', '치밀하', '경쾌하', '알맞', '긴밀하', '달달하', '터무니없', '미묘하', '다급하', '차분하', '굳건하', '넉넉하', '빽빽하', '근소하', '방대하', '막중하', '요렇', '과감하', '높아지', '지루하', '심하', '붉', '기괴하', '간편하', '화려하', '드넓', '고고하', '시급하', '우연찮', '참신하

In [283]:
unique_nj_va_keyword

{'가쁘',
 '가파르',
 '간단하',
 '간략하',
 '간절하',
 '간편하',
 '강렬하',
 '걸맞',
 '견고하',
 '경쾌하',
 '고고하',
 '고르',
 '고소하',
 '공고하',
 '공공연하',
 '과감하',
 '과하',
 '광활하',
 '괜하',
 '교묘하',
 '구성지',
 '구수하',
 '굳건하',
 '굴곡지',
 '굵직하',
 '궁금하',
 '그',
 '그러하',
 '그럴싸하',
 '그렇',
 '극심하',
 '근사하',
 '근소하',
 '기괴하',
 '기발하',
 '긴밀하',
 '깊',
 '깐깐하',
 '깜찍하',
 '깨끗하',
 '꾸준하',
 '끄',
 '끈끈하',
 '끝없',
 '나른하',
 '난감하',
 '난데없',
 '난처하',
 '난해하',
 '낮',
 '넉넉하',
 '넓',
 '높',
 '높아지',
 '누렇',
 '느긋하',
 '느끼하',
 '느슨하',
 '능수능란하',
 '능숙하',
 '늦',
 '다급하',
 '다름없',
 '단단하',
 '단호하',
 '달',
 '달달하',
 '달콤하',
 '담백하',
 '답답하',
 '대견하',
 '대단하',
 '더디',
 '도도하',
 '뒤늦',
 '드넓',
 '드물',
 '든든하',
 '따끈하',
 '따듯하',
 '따뜻하',
 '따스하',
 '딱딱하',
 '또렷하',
 '똑같',
 '뚜렷하',
 '뜨끈하',
 '뜻깊',
 '마땅하',
 '막대하',
 '막중하',
 '맛나',
 '매콤하',
 '머지않',
 '멀',
 '멋쩍',
 '명확하',
 '목마르',
 '못지않',
 '무난하',
 '무색하',
 '묽',
 '미묘하',
 '미미하',
 '미약하',
 '밋밋하',
 '바람직하',
 '발랄하',
 '발칙하',
 '밤늦',
 '방대하',
 '별다르',
 '복잡하',
 '볼메',
 '부르',
 '부질없',
 '불가피하',
 '불쌍하',
 '붉',
 '비릿하',
 '비슷하',
 '비장하',
 '빡빡하',
 '빵빵하',
 '빽빽하',
 '뻑뻑하',
 '뻔뻔하',
 '뿌듯하',
 '삐뚤',
 '상관

### VA - 블랙핑크 전체 vs meaningword 키워드 비교

In [286]:
bp_va_keyword = []

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA']] #형용사
    for idx, word in enumerate(tmp) :
        bp_va_keyword.append(word)


In [289]:
bp_va_meaningwords =[]

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['VA'] and token.form in meaning_words] #형용사
    for idx, word in enumerate(tmp) :
        bp_va_meaningwords.append(word)

In [291]:
len(bp_va_meaningwords)

3190

In [292]:
#3월치 블랙핑크 뉴스 + 블로그 형용사 리스트

len(set(bp_va_keyword)), len(set(bp_va_meaningwords)) #약 100개가 걸러짐

(316, 128)

In [295]:
# 각 리스트를 집합(set)으로 변환
bp_va_set_keyword = set(bp_va_keyword)
bp_va_set_meaningwords = set(bp_va_meaningwords)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = bp_va_set_keyword.intersection(bp_va_set_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'분하', '어리', '거세', '까맣', '길', '빼어나', '연하', '유하', '편하', '섹시하', '벌겋', '환하', '진하', '차', '크', '얇', '짧', '기차', '슬프', '재미있', '별나', '당차', '해맑', '눈부시', '뜸하', '빨갛', '빈틈없', '어질', '앙증맞', '변함없', '여리', '멍하', '무르', '아리', '싸', '많', '강하', '싸하', '잘생기', '힘들', '올바르', '어리석', '짓궂', '느리', '예쁘', '묘하', '속상하', '손색없', '장하', '서투르', '친하', '멋있', '엄청나', '찐하', '낯설', '새하얗', '굵', '반하', '급하', '바쁘', '밝', '젊', '뛰어나', '나쁘', '멋지', '격하', '검', '파랗', '가녀리', '약하', '동떨어지', '값지', '다르', '특별나', '비싸', '괜찮', '끊임없', '뻔하', '귀하', '몽글', '궂', '맛있', '하얗', '철없', '노랗', '재밌', '낡', '활기차', '커다랗', '똑바르', '어이없', '끈질기', '힘없', '꾸밈없', '건방지', '쎄', '네모나', '빠르', '아프', '좋', '기쁘', '세', '배부르', '관계없', '폭넓', '이쁘', '거침없', '작', '지나치', '싫', '야하', '흔하', '질기', '낯익', '가느다랗', '남다르', '색다르', '바르', '맑', '못생기', '거칠', '배고프', '흐리', '마르', '로맨틱하', '야무지', '착하', '희'}
공통 키워드 개수: 128


In [296]:
unique_nj_va_keyword = bp_va_set_keyword.difference(common_keywords)
unique_nj_va_meaningwords = bp_va_set_meaningwords.difference(common_keywords)

print("기사 키워드 리스트 :", unique_nj_va_keyword) # 200, 300
print("형태소 키워드 리스트 :", unique_nj_va_meaningwords)


기사 키워드 리스트 : {'도도하', '탄탄하', '드물', '저렴하', '복잡하', '멀쩡하', '화기애애하', '튼튼하', '선선하', '어색하', '상큼하', '생생하', '익숙하', '우아하', '잦', '맛나', '화창하', '친근하', '습하', '쏠쏠하', '과하', '어리둥절하', '상관없', '별다르', '관대하', '신선하', '저렇', '묽', '수많', '솔깃하', '향긋하', '유창하', '심심하', '정교하', '좋아하', '고소하', '단호하', '달콤하', '솔직하', '핫하', '답답하', '날씬하', '어딨', '뚜렷하', '미미하', '화사하', '날렵하', '단단하', '웬만하', '지난하', '열띠', '난감하', '확실하', '걸맞', '대단하', '깊', '가파르', '투박하', '헐렁하', '묵직하', '충분하', '납작하', '깜찍하', '쫀득하', '머나멀', '이러하', '쿨하', '섬세하', '기나길', '새콤하', '더디', '평범하', '단정하', '달', '무난하', '느끼하', '경쾌하', '알맞', '달달하', '굳건하', '넉넉하', '근소하', '요렇', '과감하', '높아지', '지루하', '쾌활하', '심하', '붉', '간편하', '화려하', '고고하', '우연찮', '각지', '무관하', '깨끗하', '뿌듯하', '알차', '시원하', '적당하', '뜨끈하', '따뜻하', '나른하', '흐릿하', '이렇', '진지하', '어떻', '쓰', '짙', '강렬하', '멀', '매끈하', '쾌적하', '수두룩하', '촘촘하', '뻔뻔하', '진정하', '담담하', '절친하', '확고하', '자세하', '신기하', '있', '문제없', '든든하', '마지못하', '너르', '수월하', '옅', '끝없', '흐뭇하', '예민하', '굉장하', '푸르', '뒤늦', '늦', '못지않', '가쁘', '훈훈하', '그럴싸하', '꾸준하', '여전하', '이러', '탁월하', '난데없', '소중하', '어떠하

- 뉴진스

In [297]:
len(set(nj_va_keyword)), len(set(nj_va_meaningwords)) #약 100개가 걸러짐

(449, 165)

- 블랙핑크

In [300]:
len(set(bp_va_keyword)), len(set(bp_va_meaningwords)) #약 100개가 걸러짐

(316, 128)

In [301]:
total_va_keyword = set(nj_va_keyword + bp_va_keyword)

In [302]:
total_va_meaningwords = set(nj_va_meaningwords+bp_va_meaningwords)

In [303]:
common_keywords = total_va_keyword.intersection(total_va_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'분하', '어리', '거세', '한결같', '드세', '빼어나', '길', '힘차', '까맣', '멋들어지', '부시', '새빨갛', '연하', '유하', '냉하', '편하', '뿌옇', '섹시하', '벌겋', '환하', '조그맣', '진하', '얇', '크', '차', '짧', '기차', '슬프', '둥글', '재미있', '별나', '당차', '해맑', '눈부시', '뜸하', '빨갛', '빈틈없', '순하', '어질', '변함없', '앙증맞', '독하', '후하', '여리', '멍하', '가늘', '무르', '아리', '싸', '틀리', '탁하', '많', '강하', '싸하', '잘생기', '힘들', '올바르', '정신없', '어리석', '짓궂', '예쁘', '세차', '우렁차', '묘하', '속상하', '간드러지', '손색없', '장하', '서투르', '친하', '멋있', '벅차', '값비싸', '엄청나', '찐하', '동그랗', '낯설', '새하얗', '굵', '잘', '반하', '급하', '잘나', '도드라지', '바쁘', '젊', '밝', '뛰어나', '나쁘', '앳되', '멋지', '뽀얗', '격하', '옹골차', '검', '파랗', '푸르르', '기운차', '굳세', '가녀리', '약하', '값지', '동떨어지', '다르', '쓸데없', '능글맞', '특별나', '새까맣', '비싸', '괜찮', '희', '뻔하', '끊임없', '귀하', '몽글', '궂', '맛있', '희망차', '짙푸르', '서글프', '짠하', '하얗', '철없', '노랗', '재밌', '낡', '활기차', '커다랗', '똑바르', '어이없', '끈질기', '힘없', '꾸밈없', '건방지', '쎄', '서툴', '모질', '네모나', '재미없', '빠르', '아프', '둔하', '세', '좋', '기쁘', '역하', '배부르', '쓰라리', '관계없', '폭넓', '오래', '기막히', '이쁘', '거침없', '만만찮', '작', '싫', '지나치', '귀찮'

In [304]:
total_unique_va_keyword = total_va_keyword.difference(common_keywords)
total_unique_va_meaningwords = total_va_meaningwords.difference(common_keywords)

print("기사+블로그 키워드 리스트 :", total_unique_va_keyword) # 200, 300
print("kkm 형태소 키워드 리스트 :", total_unique_va_meaningwords)


기사+블로그 키워드 리스트 : {'탄탄하', '풋풋하', '조급하', '틀림없', '선선하', '상큼하', '싸늘하', '잦', '맛나', '화창하', '쏠쏠하', '어리둥절하', '별다르', '심심하', '세세하', '찰지', '단호하', '시큰둥하', '답답하', '어딨', '뚜렷하', '웅장하', '소소하', '읎', '비릿하', '걸맞', '생소하', '위대하', '납작하', '밤늦', '깜찍하', '섬세하', '난처하', '더디', '구성지', '단정하', '어눌하', '긴밀하', '달달하', '터무니없', '미묘하', '다급하', '간편하', '화려하', '열렬하', '공고하', '이렇', '절실하', '어떻', '쓰', '따듯하', '그러하', '불쌍하', '발칙하', '수두룩하', '진정하', '든든하', '능숙하', '푸르', '뒤늦', '가쁘', '그럴싸하', '섭섭하', '느긋하', '혹독하', '광활하', '정갈하', '따스하', '완연하', '또렷하', '굴곡지', '끄', '급격하', '간단하', '은은하', '비리', '밋밋하', '수수하', '승하', '삐뚤', '흥미진진하', '막대하', '크나크', '똑같', '고르', '아낌없', '풍부하', '오지', '높', '공공연하', '퍽퍽하', '무성하', '도도하', '자잘하', '화기애애하', '튼튼하', '매콤하', '생생하', '난해하', '친근하', '습하', '무색하', '짜릿하', '과하', '괜하', '조그마하', '저렇', '잠잠하', '수많', '따끈하', '아득하', '정교하', '핫하', '화사하', '아슬하', '부르', '날렵하', '단단하', '열띠', '확실하', '가파르', '묵직하', '쫀득하', '기나길', '새콤하', '달', '서운하', '무난하', '느끼하', '굳건하', '넉넉하', '빽빽하', '방대하', '막중하', '쾌활하', '심하', '붉', '기괴하', '드넓', '뿌듯하', '자욱하', '적당하', '흐릿하', '진지하', '하'

#### 추가 필요 키워드들

In [ ]:
'강렬하', '견고하', '경쾌하', '고고하', '과감하','관대하','굉장하','교묘하','구수하','굳건하','굴곡지','굵직하','궁금하','근사하','기괴하',
'기발하','깊','깐깐하','깜찍하','깨끗하','꾸준하','나른하','난감하','난데없','난해하','날렵하','날씬하','납작하','넉넉하','높아지','누렇','느긋하',
'느끼하','느슨하','능수능란하','능숙하','단단하','단정하','단호하','달','달달하','달콤하','담담하','담백하','답답하','대견하',
 '대단하',
 '더디',
 '도도하',
 '드물',
 '든든하',
 '따끈하',
 '따듯하',
 '따뜻하',
 '따스하',
 '딱딱하',
 '또렷하',
 '뚜렷하',
 '뜨끈하',
 '막대하',
 '매끈하',
 '매콤하',
 '머나멀',
 '멀쩡하',
 '명확하',
 '무난하',
 '묵직하',
 '문제없',
 '묽',
 '미묘하',
 '미미하',
 '미약하',
 '밋밋하',
 '바람직하',
 '발랄하',
 '발칙하',
 '별다르',
 '복잡하',
 '볼메',
 '부적절하',
 '불쌍하',
 '붉',
 '비리',
 '비릿하',
 '비장하',
 '빡빡하',
 '빵빵하',
 '빽빽하',
 '뻑뻑하',
 '뻔뻔하',
 '뿌듯하',
 '삐뚤',
 '상세하',
 '상큼하',
 '새콤하',
 '생뚱맞',
 '생생하',
 '생소하',
 '서운하',
 '선선하',
 '설',
 '섬세하',
 '섭섭하',
 '성하',
 '세세하',
 '세심하',

#### XR 유의미 단어


In [305]:
train= pd.read_csv(f"{DATA_PATH}XR_kkm_세종말뭉치_통계조회_.csv")

In [ ]:
df1 = train.iloc[:400]
df1['형태소'].tolist()

In [ ]:
df2 = train.iloc[1000:1200]
df2['형태소'].tolist()

In [ ]:
train['형태소'].tolist()

In [307]:
xr_meaningwords = [
 '강력',
 '훌륭',
 '바람직',
 '따뜻',
 '깨끗',
 '화려',
 '궁금',
 '진정',
 '진지',
 '대단',
 '익숙',
 '다양',
 '조용',
 '시원',
 '답답',
 '소중',
 '활발',
 '엉뚱',
 '평범',
 '엄격',
 '솔직',
 '불쌍',
 '어색',
 '억울',
 '절실',
 '쓸쓸',
 '생생',
 '민감',
 '신선',
 '강렬',
 '만만',
 '끔찍',
 '지루',
 '소박',
 '이상',
 '단단',
 '격렬',
 '한심',
 '초라',
 '과감',
 '튼튼',
 '유명',
 '유연',
 '섬세',
 '똑똑',
 '단호',
 '지독',
 '섭섭',
 '차분',
 '간절',
 '순진',
 '애매',
 '심심',
 '불쾌',
 '절박',
 '은밀',
 '예민',
 '깔끔',
 '싱싱',
 '달콤',
 '잔잔',
 '유쾌',
 '서늘',
 '따스',
 '싸늘',
 '집요',
 '성급',
 '황당',
 '떳떳',
 '유치',
 '지저분',
 '우아',
 '과격',
 '거룩',
 '위태',
 '거대',
 '단정',
 '느긋',
 '용감',
 '저렴',
 '허전',
 '든든',
 '창백',
 '멍청',
 '독특',
 '조급',
 '급급',
 '고귀',
 '생소',
 '희한',
 '친근',
 '어수선',
 '철저',
 '거북',
 '썰렁',
 '담담',
 '세심',
 '경쾌',
 '허술',
 '원만',
 '예리',
 '야릇',
 '비겁',
 '근사',
 '상쾌',
 '허름',
 '진부',
 '씩씩',
 '반듯',
 '비장',
 '포근',
 '고약',
 '고상',
 '미숙',
 '정중',
 '웅장',
 '아찔',
 '난해',
 '안이',
 '냉혹',
 '못마땅',
 '열렬',
 '고단',
 '수상',
 '탄탄',
 '강인',
 '뻔뻔',
 '연약',
 '아늑',
 '자그마',
 '추악',
 '앙상',
 '관대',
 '묵직',
 '치열',
 '엉성',
 '큼직',
 '순박',
 '구수',
 '화사',
 '험악',
 '일관',
 '광활',
 '애틋',
 '끈끈',
 '자상',
 '무시무시',
 '삭막',
 '경건',
 '발랄',
 '의젓',
 '착실',
 '꾸준',
 '쌀쌀',
 '왜소',
 '처참',
 '숭고',
 '날씬',
 '상냥',
 '교활',
 '처량',
 '산뜻',
 '영리',
 '납작',
 '산만',
 '무뚝뚝',
 '애절',
 '진솔',
 '얄팍',
 '뻣뻣',
 '담백',
 '아련',
 '오묘',
 '훈훈',
 '말끔',
 '온화',
 '침울',
 '어정쩡',
 '신랄',
 '공손',
 '냉철',
 '날렵',
 '아담',
 '해괴',
 '투박',
 '민첩',
 '육중',
 '우람',
 '외람',
 '묵묵',
 '근소',
 '영특',
 '아둔',
 '괴팍',
 '거무스름',
 '뾰로통',
 '예쁘장',
 '졸렬',
 '온건',
 '빵빵',
 '담대',
 '따듯',
 '다정다감',
 '께름칙',
 '핼쑥',
 '흉포',
 '따끈',
 '달착지근',
 '서글서글',
 '어리벙벙',
 '현란',
 '홀쭉',
 '멀뚱',
 '공명정대',
 '유약',
 '끈적',
 '위태위태',
 '동글납작',
 '팔팔',
 '소소',
 '살벌',
 '순진무구',
 '믿음직',
 '허약',
 '느릿',
 '주도면밀',
 '둥그스름',
 '거무스레',
 '소탈',
 '미적지근',
 '호쾌',
 '걸걸',
 '누르스름',
 '우스꽝',
 '호화',
 '동그스름',
 '치명',
 '심약',
 '파릇',
 '저속',
 '영민',
 '발칙',
 '명랑',
 '미련',
 '똘똘',
 '새콤',
 '완고',
 '굵직',
 '소심',
 '갑갑',
 '당돌',
 '당당',
 '늘씬',
 '우수',
 '용',
 '미진',
 '빽빽',
 '의연',
 '매끈',
 '음울',
 '합리',
 '척박',
 '근엄',
 '면밀',
 '상큼',
 '노련',
 '건전',
 '강경',
 '푸근',
 '풍족',
 '가련',
 '괘씸',
 '조잡',
 '시들',
 '세련',
 '헐렁',
 '누추',
 '자잘',
 '활달',
 '적대',
 '완연',
 '늠름',
 '스산',
 '숙연',
 '얼떨떨',
 '어엿',
 '자신만만',
 '어두컴컴',
 '갸름',
 '쾌활',
 '고지식',
 '방만',
 '경박',
 '건장',
 '꼿꼿',
 '따분',
 '초췌',
 '부당',
 '왕성',
 '기특',
 '미지근',
 '건실',
 '밋밋',
 '어렴풋',
 '길쭉',
 '도도',
 '향긋',
 '구태의연',
 '초연',
 '정갈',
 '호젓',
 '수려',
 '솔깃',
 '준엄',
 '시큰둥',
 '침침',
 '꿋꿋',
 '정결',
 '번듯',
 '단아',
 '캄캄',
 '불온',
 '시무룩',
 '빳빳',
 '비열',
 '지긋',
 '꺼림칙',
 '음탕',
 '영악',
 '야비',
 '냉랭',
 '노골',
 '따사',
 '비범',
 '찜찜',
 '대범',
 '어눌',
 '흥미진진',
 '어둑',
 '영롱',
 '조촐',
 '절절',
 '깜찍',
 '푸르스름',
 '세세',
 '절친',
 '풍만',
 '낭자',
 '유용',
 '칙칙',
 '쟁쟁',
 '기민',
 '대담',
 '수수',
 '심드렁',
 '진귀',
 '얼얼',
 '말쑥',
 '음침',
 '삐딱',
 '옹색',
 '추잡',
 '뻐근',
 '장대',
 '풋풋',
 '훤칠',
 '결연',
 '냉엄',
 '뻔',
 '비릿',
 '인자',
 '유력',
 '방자',
 '가혹',
 '다정',
 '괴이',
 '야트막',
 '화창',
 '애잔',
 '고고',
 '굳건',
 '갸륵',
 '매정',
 '아리송',
 '취약',
 '덤덤',
 '온순',
 '심란',
 '조악',
 '장중',
 '농후',
 '넓적',
 '울적',
 '해박',
 '둔탁',
 '무고',
 '까무잡잡',
 '무지막지',
 '촘촘',
 '원숙',
 '느끼',
 '능통',
 '호탕',
 '휘황',
 '음험',
 '황당무계',
 '흉흉',
 '퀴퀴',
 '강직',
 '으슥',
 '격',
 '저돌', #
 '잔인',
 '음습',
 '뚱뚱',
 '말짱',
 '빡빡',
 '괴상',
 '푹신',
 '원대',
 '고즈넉',
 '꾀죄죄',
 '반반',
 '깐깐',
 '눅눅',
 '독실',
 '기구',
 '망측',
 '처연',
 '이슥',
 '선량',
 '수척',
 '황홀',
 '낭랑',
 '싹싹',
 '맹랑',
 '명석',
 '꼼꼼',
 '야박',
 '듬직',
 '고매',
 '걸쭉',
 '가뿐',
 '유익',
 '후줄근',
 '어수룩',
 '황폐',
 '유순',
 '특출',
 '통렬',
 '고리타분',
 '우직',
 '섹시',
 '견실',
 '강대',
 '무정',
 '중후',
 '깜깜',
 '비천',
 '두둑',
 '둔중',
 '순전',
 '희끄무레',
 '구부정',
 '무덤덤',
 '유복',
 '은근',
 '불미',
 '망연',
 '도톰',
 '까칠',
 '요란',
 '평이',
 '청명',
 '퀭',
 '과',
 '옹졸',
 '너저분',
 '파리',
 '무던',
 '묘연',
 '완곡',
 '청아',
 '나약',
 '능란',
 '초조',
 '무모',
 '엉큼',
 '어둠침침',
 '혹심',
 '처절',
 '깍듯',
 '알뜰',
 '알량',
 '거무튀튀',
 '고루',
 '다소곳',
 '가무잡잡',
 '청량',
 '쾌청',
 '정밀',
 '질겁',
 '아뜩',
 '아릿',
 '간명',
 '으리으리',
 '정정당당',
 '청초',
 '강건',
 '뜨악',
 '망망',
 '등등',
 '괴괴',
 '난잡',
 '간사',
 '파르스름',
 '휘황찬란',
 '용의주도',
 '부지런',
 '불그레',
 '뜨뜻',
 '진득',
 '진중',
 '범상',
 '펑퍼짐',
 '고만고만',
 '편편',
 '출중',
 '감미',
 '소담',
 '병약',
 '성성',
 '각별',
 '공고',
 '과대',
 '가증',
 '일천',
 '왁자',
 '유려',
 '먹먹',
 '미천',
 '미려', #1000~1200개
 '과도',
 '괄괄',
 '기기묘묘',
 '터프',
 '앙증',
 '멀쑥',
 '딴딴',
 '궁벽',
 '고결',
 '고아',
 '시시껄렁',
 '어두침침',
 '이상야릇',
 '형형',
 '혼탁',
 '쩨쩨',
 '찝찝',
 '저릿',
 '신성',
 '거무죽죽',
 '휑뎅그렁',
 '노르스름',
 '능수능란',
 '불그죽죽',
 '편평',
 '해쓱',
 '침통',
 '털털',
 '토실',
 '지순',
 '지엄',
 '우울',
 '울울',
 '신속',
 '심난',
 '심원',
 '살뜰',
 '뜨끔',
 '마뜩',
 '망극',
 '가열',
 '찌뿌드드',
 '호화찬란',
 '득의만만',
 '로맨틱',
 '어리숙',
 '에로틱',
 '현격',
 '쫄깃',
 '장쾌',
 '적의',
 '소홀',
 '앙큼',
 '애처',
 '박정',
 '범연',
 '부박',
 '명철',
 '무결',
 '다기',
 '교교',
 '극미',
 '추',
 '푸르뎅뎅',
 '푸르죽죽',
 '감',
 '드라마틱',
 '뻑적지근',
 '사근사근',
 '애매모호',
 '어머어마',
 '무지근',
 '오종종',
 '이쁘장',
 '황막',
 '흉악',
 '컬컬',
 '소략',
 '소복',
 '심장',
 '양순',
 '무감',
 '무망',
 '껌껌',
 '발그스름',
 '간구',
 '덥수룩',
 '도도록',
 '부정직',
 '편벽',
 '희끗',
 '척척',
 '친숙',
 '자별',
 '정대',
 '정숙',
 '엔간',
 '오롯',
 '완서',
 '위중',
 '서름',
 '선뜻',
 '막강',
 '망칙',
 '냉정',
 '노숙',
 '우두망찰',
 '흐리멍덩',
 '고요',
 '거만',
 '암울',
 '괴상망측',
 '두리뭉실',
 '둥글넓적',
 '불그스름',
 '불효막심',
 '살기등등',
 '시금털털',
 '그들먹',
 '판판',
 '훤출',
 '천진',
 '측은',
 '정치',
 '역연',
 '요상',
 '유일',
 '숭엄',
 '식상',
 '방정',
 '분망',
 '말똥',
 '무계',
 '괄목',
 '간악',
 '걸찍',
 '고만',
 '뜨뜻미지근',
 '쇠']

### XR - 뉴진스 전체 vs meaningword 키워드 비교

In [308]:
nj_xr_keyword = []

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR']]
    for idx, word in enumerate(tmp) :
        nj_xr_keyword.append(word)


In [309]:
len(nj_xr_keyword)

5209

In [310]:
nj_xr_meaningwords =[]

for idx, text in enumerate(nj_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR'] and token.form in xr_meaningwords] #형용사
    for idx, word in enumerate(tmp) :
        nj_xr_meaningwords.append(word)

In [311]:
#3월치 뉴진스 블로그 형용사 리스트

len(set(nj_xr_keyword)), len(set(nj_xr_meaningwords)) #약 100개가 걸러짐

(472, 253)

In [332]:
# 각 리스트를 집합(set)으로 변환
nj_xr_set_keyword = set(nj_xr_keyword)
nj_xr_set_meaningwords = set(nj_xr_meaningwords)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = nj_xr_set_keyword.intersection(nj_xr_set_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'진지', '궁금', '솔깃', '심드렁', '관대', '천진', '펑퍼짐', '취약', '못마땅', '야비', '둔탁', '차분', '고상', '달콤', '출중', '유쾌', '불쌍', '생소', '격렬', '만만', '진정', '끔찍', '싸늘', '눅눅', '단정', '세심', '다정', '담담', '진중', '서늘', '자상', '단아', '유명', '뻔뻔', '간절', '상큼', '비장', '신선', '독특', '매끈', '민감', '매정', '근사', '능수능란', '탄탄', '조악', '길쭉', '집요', '난해', '당돌', '의젓', '유일', '완곡', '조용', '막강', '섹시', '다양', '살뜰', '느긋', '예민', '훈훈', '순진무구', '수수', '치열', '황홀', '가련', '애매모호', '대담', '아련', '불쾌', '노련', '따듯', '민첩', '감미', '잔잔', '청량', '강력', '화려', '왕성', '섬세', '갑갑', '미려', '대단', '빡빡', '당당', '섭섭', '투박', '익숙', '은밀', '살벌', '과감', '담백', '구수', '어엿', '애잔', '면밀', '공고', '씩씩', '기특', '드라마틱', '우울', '수상', '은근', '깜찍', '걸쭉', '쫄깃', '지독', '온순', '포근', '쟁쟁', '큼직', '갸름', '활발', '깐깐', '냉혹', '느끼', '엄격', '풍족', '훌륭', '따스', '친근', '상냥', '강인', '저렴', '굳건', '유약', '오묘', '날씬', '똑똑', '청초', '밋밋', '사근사근', '고즈넉', '기민', '경건', '소중', '조잡', '생생', '과격', '미숙', '야릇', '방만', '꾸준', '납작', '억울', '유연', '친숙', '일관', '휘황찬란', '애틋', '진솔', '까칠', '시원', '영민', '튼튼', '향긋', '지루', '새콤', '촘촘', '가혹', '아찔', '늘씬', '영리

In [333]:
unique_nj_xr_keyword = nj_xr_set_keyword.difference(common_keywords)
unique_nj_xr_meaningwords = nj_xr_set_meaningwords.difference(common_keywords)

print("기사+블로그 키워드 리스트 :", unique_nj_xr_keyword)
print("kkm xr 키워드 리스트 :", unique_nj_xr_meaningwords)


기사+블로그 키워드 리스트 : {'마땅', '주요', '쏠쏠', '특이', '웬만', '무방', '심오', '특유', '머쓱', '심상', '가능', '적당', '정교', '어떡', '심각', '상이', '곡진', '으르렁', '난감', '떠들썩', '극명', '마다', '위대', '잡다', '희귀', '조그마', '간편', '수두룩', '복잡다단', '성대', '분분', '빈번', '심', '딱딱', '아스라', '어마어마', '시급', '한가', '또렷', '복잡', '아른', '광범', '온전', '흔', '짤막', '깊숙', '평평', '친밀', '본격', '자명', '다분', '정연', '절묘', '통쾌', '불사', '풍성', '상세', '미세', '치밀', '탁월', '촉촉', '타당', '팽팽', '고소', '기이', '긴밀', '당연', '무관', '은은', '극진', '면', '명쾌', '획일', '지극', '신기', '무도', '어지간', '민망', '뿌듯', '단출', '대견', '까마득', '한산', '미묘', '씁쓸', '거창', '모호', '후덥지근', '특별', '세밀', '괜', '넉넉', '단조', '뜸', '참신', '간략', '시끌', '껑충', '진기', '톡톡', '화기애애', '떨떠름', '어중간', '여의', '뒤숭숭', '돈독', '서운', '환', '굉장', '공공연', '따끔', '비롯', '간소', '나른', '합당', '능숙', '혹독', '간단', '확연', '말랑', '정정', '무난', '명확', '큼지막', '맞이', '선명', '맹렬', '잠잠', '순탄', '쾌적', '매콤', '찬란', '뚜렷', '불가피', '화', '유사', '희박', '기발', '뭉근', '막론', '기괴', '유창', '충분', '간결', '흐뭇', '의아', '온당', '여전', '청청', '두근', '방대', '미미', '확실', '임박', '폭신', '술렁', '지대', '막중', '험난', '이러', '개운', '널

### XR - 블랙핑크 전체 vs meaningword 키워드 비교

In [334]:
bp_xr_keyword = []

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR']] #형용사
    for idx, word in enumerate(tmp) :
        bp_xr_keyword.append(word)


In [335]:
len(bp_xr_keyword)

1840

In [336]:
bp_xr_meaningwords =[]

for idx, text in enumerate(bp_text) :
    result = kiwi.tokenize(text,stopwords=stopwords)
    tmp = [token.form for token in result if token.tag in ['XR'] and token.form in xr_meaningwords] #형용사
    for idx, word in enumerate(tmp) :
        bp_xr_meaningwords.append(word)

In [337]:
len(bp_xr_meaningwords)

1133

In [338]:
#3월치 뉴진스 블로그 형용사 리스트

len(set(bp_xr_keyword)), len(set(bp_xr_meaningwords)) #약 100개가 걸러짐

(319, 172)

In [339]:
# 각 리스트를 집합(set)으로 변환
bp_xr_set_keyword = set(bp_xr_keyword)
bp_xr_set_meaningwords = set(bp_xr_meaningwords)

# 공통 키워드: 모든 리스트에 있는 키워드
common_keywords = bp_xr_set_keyword.intersection(bp_xr_set_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'허전', '궁금', '진지', '도톰', '원만', '차분', '달콤', '출중', '유쾌', '생소', '진정', '만만', '근소', '끔찍', '싸늘', '단정', '세심', '다정', '담담', '서늘', '절친', '유명', '뻔뻔', '단아', '엉성', '간절', '상큼', '신선', '독특', '매끈', '민감', '근사', '탄탄', '집요', '난해', '당돌', '다소곳', '완곡', '조용', '섹시', '다양', '예민', '훈훈', '수수', '황홀', '불쾌', '따듯', '잔잔', '청량', '강력', '화려', '섬세', '예쁘장', '대단', '빡빡', '음탕', '당당', '익숙', '구수', '은밀', '과감', '담백', '추잡', '공고', '심란', '드라마틱', '수상', '로맨틱', '깜찍', '지독', '포근', '쟁쟁', '큼직', '소심', '활발', '느끼', '훌륭', '풍족', '엄격', '강인', '저렴', '굳건', '오묘', '날씬', '똑똑', '청초', '밋밋', '고즈넉', '소중', '조잡', '생생', '야릇', '꾸준', '억울', '유연', '의연', '애틋', '진솔', '까칠', '시원', '향긋', '지루', '새콤', '촘촘', '아찔', '늘씬', '찝찝', '영리', '어색', '가뿐', '깔끔', '청명', '처연', '허술', '도도', '초연', '어수선', '유치', '산뜻', '노골', '세련', '애매', '평범', '화창', '소소', '날렵', '청아', '용감', '어리숙', '끈끈', '희한', '든든', '황당', '바람직', '냉철', '위태', '푹신', '깨끗', '앙상', '열렬', '지저분', '엉뚱', '영롱', '솔직', '털털', '답답', '평이', '침울', '졸렬', '따뜻', '화사', '발랄', '강렬', '우아', '각별', '묵직', '어정쩡', '자그마', '담대', '아담', '심심', '강건', '웅장', '

In [340]:
unique_bp_xr_keyword = bp_xr_set_keyword.difference(common_keywords)
unique_bp_xr_meaningwords = bp_xr_set_meaningwords.difference(common_keywords)

print("기사+블로그 키워드 리스트 :", unique_bp_xr_keyword)
print("kkm xr 키워드 리스트 :", unique_bp_xr_meaningwords)


기사+블로그 키워드 리스트 : {'마땅', '멸', '주요', '쏠쏠', '무방', '웬만', '특이', '머쓱', '심상', '가능', '적당', '무성', '떠들석', '긴가민가', '어떡', '심각', '상이', '한적', '난감', '쌉쌀', '뭉클', '마다', '의미심장', '위대', '통통', '성대', '분분', '빈번', '심', '어마어마', '또렷', '복잡', '무근', '흔', '가지런', '짤막', '친밀', '본격', '절묘', '통쾌', '달짝지근', '풍성', '상세', '명징', '탁월', '미세', '촉촉', '분주', '기이', '고소', '긴밀', '텁텁', '팔랑', '당연', '축축', '은은', '극진', '쫑알', '신기', '민망', '뿌듯', '씁쓸', '모호', '특별', '겸허', '괜', '넉넉', '비등', '흐릿', '참신', '시끌', '껑충', '화기애애', '널널', '엄', '뒤숭숭', '돈독', '서운', '환', '굉장', '비롯', '그윽', '비슷비슷', '빠듯', '간단', '확연', '말랑', '맞이', '무난', '명확', '선명', '자발', '잠잠', '순탄', '쾌적', '찬란', '뚜렷', '불가피', '희박', '유창', '충분', '흐뭇', '여전', '청청', '방대', '미미', '확실', '임박', '폭신', '비아냥', '파다', '이러', '널찍', '어떠', '방불', '짜릿', '무수', '불구', '강', '푸짐', '적절', '즐비', '정확', '귀중', '죄송', '공교', '그러', '단순', '중', '편', '동일', '두리번', '경미', '막대', '미약', '다채', '용이', '자세', '교묘', '명백', '확고', '풍부', '수월', '테이크', '막막', '생경', '비슷'}
kkm xr 키워드 리스트 : set()


## 형용사VA + 어근XR 리스트




### VA

- 뉴진스

In [341]:
len(set(nj_va_keyword)), len(set(nj_va_meaningwords)),

(449, 165)

- 블랙핑크

In [342]:
len(set(bp_va_keyword)), len(set(bp_va_meaningwords)),

(316, 128)

In [343]:
total_va_keyword = set(nj_va_keyword + bp_va_keyword)
total_va_meaningwords = set(nj_va_meaningwords+bp_va_meaningwords)

In [344]:
common_keywords = total_va_keyword.intersection(total_va_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'분하', '어리', '거세', '한결같', '드세', '빼어나', '길', '힘차', '까맣', '멋들어지', '부시', '새빨갛', '연하', '유하', '냉하', '편하', '뿌옇', '섹시하', '벌겋', '환하', '조그맣', '진하', '얇', '크', '차', '짧', '기차', '슬프', '둥글', '재미있', '별나', '당차', '해맑', '눈부시', '뜸하', '빨갛', '빈틈없', '순하', '어질', '변함없', '앙증맞', '독하', '후하', '여리', '멍하', '가늘', '무르', '아리', '싸', '틀리', '탁하', '많', '강하', '싸하', '잘생기', '힘들', '올바르', '정신없', '어리석', '짓궂', '예쁘', '세차', '우렁차', '묘하', '속상하', '간드러지', '손색없', '장하', '서투르', '친하', '멋있', '벅차', '값비싸', '엄청나', '찐하', '동그랗', '낯설', '새하얗', '굵', '잘', '반하', '급하', '잘나', '도드라지', '바쁘', '젊', '밝', '뛰어나', '나쁘', '앳되', '멋지', '뽀얗', '격하', '옹골차', '검', '파랗', '푸르르', '기운차', '굳세', '가녀리', '약하', '값지', '동떨어지', '다르', '쓸데없', '능글맞', '특별나', '새까맣', '비싸', '괜찮', '희', '뻔하', '끊임없', '귀하', '몽글', '궂', '맛있', '희망차', '짙푸르', '서글프', '짠하', '하얗', '철없', '노랗', '재밌', '낡', '활기차', '커다랗', '똑바르', '어이없', '끈질기', '힘없', '꾸밈없', '건방지', '쎄', '서툴', '모질', '네모나', '재미없', '빠르', '아프', '둔하', '세', '좋', '기쁘', '역하', '배부르', '쓰라리', '관계없', '폭넓', '오래', '기막히', '이쁘', '거침없', '만만찮', '작', '싫', '지나치', '귀찮'

In [345]:
total_unique_va_keyword = total_va_keyword.difference(common_keywords)
total_unique_va_meaningwords = total_va_meaningwords.difference(common_keywords)

print(len(total_unique_va_keyword),"기사+블로그 키워드 리스트 :", total_unique_va_keyword)
print(len(total_unique_va_meaningwords),"kkm VA 형태소 키워드 리스트 :", total_unique_va_meaningwords)


323 기사+블로그 키워드 리스트 : {'탄탄하', '풋풋하', '조급하', '틀림없', '선선하', '상큼하', '싸늘하', '잦', '맛나', '화창하', '쏠쏠하', '어리둥절하', '별다르', '심심하', '세세하', '찰지', '단호하', '시큰둥하', '답답하', '어딨', '뚜렷하', '웅장하', '소소하', '읎', '비릿하', '걸맞', '생소하', '위대하', '납작하', '밤늦', '깜찍하', '섬세하', '난처하', '더디', '구성지', '단정하', '어눌하', '긴밀하', '달달하', '터무니없', '미묘하', '다급하', '간편하', '화려하', '열렬하', '공고하', '이렇', '절실하', '어떻', '쓰', '따듯하', '그러하', '불쌍하', '발칙하', '수두룩하', '진정하', '든든하', '능숙하', '푸르', '뒤늦', '가쁘', '그럴싸하', '섭섭하', '느긋하', '혹독하', '광활하', '정갈하', '따스하', '완연하', '또렷하', '굴곡지', '끄', '급격하', '간단하', '은은하', '비리', '밋밋하', '수수하', '승하', '삐뚤', '흥미진진하', '막대하', '크나크', '똑같', '고르', '아낌없', '풍부하', '오지', '높', '공공연하', '퍽퍽하', '무성하', '도도하', '자잘하', '화기애애하', '튼튼하', '매콤하', '생생하', '난해하', '친근하', '습하', '무색하', '짜릿하', '과하', '괜하', '조그마하', '저렇', '잠잠하', '수많', '따끈하', '아득하', '정교하', '핫하', '화사하', '아슬하', '부르', '날렵하', '단단하', '열띠', '확실하', '가파르', '묵직하', '쫀득하', '기나길', '새콤하', '달', '서운하', '무난하', '느끼하', '굳건하', '넉넉하', '빽빽하', '방대하', '막중하', '쾌활하', '심하', '붉', '기괴하', '드넓', '뿌듯하', '자욱하', '적당하', '흐릿하', '진지하',

### XR

- 뉴진스

In [346]:
len(set(nj_xr_keyword)), len(set(nj_xr_meaningwords)),
# len(set(xr_nj_tfidf_top300)),len(set(xr_meaningwords_nj_tfidf_top300))

(472, 253)

- 블랙핑크

In [347]:
len(set(bp_xr_keyword)), len(set(bp_xr_meaningwords)),
# len(set(xr_bp_tfidf_top300)),len(set(xr_meaningwords_bp_tfidf_top300))

(319, 172)

- 총계

In [348]:
len(set(nj_xr_keyword + bp_xr_keyword)), len(set(nj_xr_meaningwords+bp_xr_meaningwords)) #

(531, 279)

In [351]:
total_xr_keyword = set(nj_xr_keyword + bp_xr_keyword)

In [352]:
total_xr_meaningwords = set(nj_xr_meaningwords+bp_xr_meaningwords)

In [353]:
common_keywords = total_xr_keyword.intersection(total_xr_meaningwords)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'허전', '궁금', '진지', '솔깃', '심드렁', '관대', '천진', '펑퍼짐', '취약', '도톰', '못마땅', '원만', '야비', '둔탁', '차분', '고상', '달콤', '출중', '유쾌', '불쌍', '생소', '격렬', '만만', '진정', '근소', '끔찍', '싸늘', '눅눅', '단정', '세심', '다정', '담담', '진중', '서늘', '자상', '단아', '유명', '뻔뻔', '간절', '절친', '엉성', '상큼', '비장', '신선', '독특', '매끈', '민감', '매정', '근사', '능수능란', '탄탄', '조악', '길쭉', '집요', '난해', '당돌', '의젓', '유일', '완곡', '조용', '다소곳', '막강', '섹시', '다양', '살뜰', '느긋', '예민', '훈훈', '순진무구', '수수', '치열', '황홀', '가련', '애매모호', '대담', '아련', '불쾌', '노련', '따듯', '민첩', '감미', '잔잔', '청량', '강력', '화려', '왕성', '섬세', '갑갑', '미려', '예쁘장', '대단', '빡빡', '당당', '섭섭', '음탕', '투박', '익숙', '은밀', '살벌', '과감', '담백', '구수', '어엿', '애잔', '추잡', '면밀', '공고', '씩씩', '심란', '기특', '드라마틱', '우울', '수상', '은근', '로맨틱', '깜찍', '걸쭉', '쫄깃', '지독', '온순', '포근', '쟁쟁', '큼직', '갸름', '소심', '활발', '깐깐', '냉혹', '느끼', '엄격', '풍족', '훌륭', '따스', '친근', '상냥', '강인', '저렴', '굳건', '유약', '오묘', '날씬', '똑똑', '청초', '밋밋', '사근사근', '고즈넉', '기민', '경건', '소중', '조잡', '생생', '과격', '미숙', '야릇', '방만', '꾸준', '납작', '억울', '유연', '의연', '친숙', '일관', '휘

In [354]:
total_unique_xr_keyword = total_xr_keyword.difference(common_keywords)
total_unique_xr_meaningwords = total_xr_meaningwords.difference(common_keywords)

print("기사+블로그 키워드 리스트 :", total_unique_va_keyword) # 200, 300
print("kkm 형태소 키워드 리스트 :", total_unique_va_meaningwords)


기사+블로그 키워드 리스트 : {'탄탄하', '풋풋하', '조급하', '틀림없', '선선하', '상큼하', '싸늘하', '잦', '맛나', '화창하', '쏠쏠하', '어리둥절하', '별다르', '심심하', '세세하', '찰지', '단호하', '시큰둥하', '답답하', '어딨', '뚜렷하', '웅장하', '소소하', '읎', '비릿하', '걸맞', '생소하', '위대하', '납작하', '밤늦', '깜찍하', '섬세하', '난처하', '더디', '구성지', '단정하', '어눌하', '긴밀하', '달달하', '터무니없', '미묘하', '다급하', '간편하', '화려하', '열렬하', '공고하', '이렇', '절실하', '어떻', '쓰', '따듯하', '그러하', '불쌍하', '발칙하', '수두룩하', '진정하', '든든하', '능숙하', '푸르', '뒤늦', '가쁘', '그럴싸하', '섭섭하', '느긋하', '혹독하', '광활하', '정갈하', '따스하', '완연하', '또렷하', '굴곡지', '끄', '급격하', '간단하', '은은하', '비리', '밋밋하', '수수하', '승하', '삐뚤', '흥미진진하', '막대하', '크나크', '똑같', '고르', '아낌없', '풍부하', '오지', '높', '공공연하', '퍽퍽하', '무성하', '도도하', '자잘하', '화기애애하', '튼튼하', '매콤하', '생생하', '난해하', '친근하', '습하', '무색하', '짜릿하', '과하', '괜하', '조그마하', '저렇', '잠잠하', '수많', '따끈하', '아득하', '정교하', '핫하', '화사하', '아슬하', '부르', '날렵하', '단단하', '열띠', '확실하', '가파르', '묵직하', '쫀득하', '기나길', '새콤하', '달', '서운하', '무난하', '느끼하', '굳건하', '넉넉하', '빽빽하', '방대하', '막중하', '쾌활하', '심하', '붉', '기괴하', '드넓', '뿌듯하', '자욱하', '적당하', '흐릿하', '진지하', '하'

## 형용사VA + 어근XR 리스트 합치고 전체 리스트 vs meaningwords 비교

- 정의 :
 - 전체 키워드 : 전체 크롤링 자료 (뉴진스, 블랙핑크의 뉴스&블로그) 의 형용사 + 어근 키워드 리스트
 - meaning words : 전체 KKM 형태소 자료의 형용사 + 어근 리스트 중 선별한 유의미 키워드(meaning words) 리스트

- 목적 ⁉
 - 전체 키워드와, meaning words를 서로 비교하여,
 - meaning words2 에 전체 키워드에는 있으나 meaning words1에 없던 키워드들 중 유의미한 것들을 추가


- 전체 키워드 리스트

In [355]:
total_nva_keyword = adjective_to_noun(total_va_keyword)
total_nva_keyword

['분함',
 '탄탄함',
 '풋풋함',
 '조급함',
 '한결같음',
 '까맣음',
 '길음',
 '틀림없음',
 '선선함',
 '상큼함',
 '싸늘함',
 '잦음',
 '맛남',
 '화창함',
 '벌겋음',
 '조그맣음',
 '쏠쏠함',
 '어리둥절함',
 '별다름',
 '재미있음',
 '눈부심',
 '심심함',
 '세세함',
 '찰짐',
 '단호함',
 '시큰둥함',
 '답답함',
 '어딨음',
 '뚜렷함',
 '웅장함',
 '소소함',
 '쌈',
 '읎음',
 '비릿함',
 '걸맞음',
 '생소함',
 '위대함',
 '납작함',
 '밤늦음',
 '깜찍함',
 '섬세함',
 '난처함',
 '묘함',
 '더딤',
 '구성짐',
 '단정함',
 '어눌함',
 '친함',
 '긴밀함',
 '달달함',
 '터무니없음',
 '미묘함',
 '다급함',
 '동그랗음',
 '새하얗음',
 '굵음',
 '도드라짐',
 '간편함',
 '화려함',
 '나쁨',
 '옹골참',
 '열렬함',
 '공고함',
 '이렇음',
 '푸르름',
 '절실함',
 '어떻음',
 '씀',
 '약함',
 '쓸데없음',
 '따듯함',
 '그러함',
 '불쌍함',
 '발칙함',
 '수두룩함',
 '진정함',
 '귀함',
 '궂음',
 '맛있음',
 '짙푸름',
 '든든함',
 '철없음',
 '능숙함',
 '재밌음',
 '푸름',
 '뒤늦음',
 '가쁨',
 '그럴싸함',
 '섭섭함',
 '느긋함',
 '쎔',
 '혹독함',
 '광활함',
 '정갈함',
 '따스함',
 '완연함',
 '또렷함',
 '좋음',
 '굴곡짐',
 '역함',
 '끔',
 '급격함',
 '간단함',
 '오램',
 '은은함',
 '비림',
 '기막힘',
 '이쁨',
 '밋밋함',
 '만만찮음',
 '수수함',
 '승함',
 '질김',
 '삐뚤음',
 '두드러짐',
 '색다름',
 '바름',
 '흥미진진함',
 '막대함',
 '방정맞음',
 '맑음',
 '크나큼',
 '똑같음',
 '배고픔',
 '거멓음',
 '고름',
 '아낌없음

In [356]:
total_nxr_keyword = xr_to_noun(total_xr_keyword)
total_nxr_keyword

['허전함',
 '멸함',
 '심드렁함',
 '웬만함',
 '취약함',
 '심상함',
 '못마땅함',
 '원만함',
 '정교함',
 '떠들석함',
 '유쾌함',
 '불쌍함',
 '생소함',
 '눅눅함',
 '다정함',
 '자상함',
 '극명함',
 '비장함',
 '매정함',
 '수두룩함',
 '분분함',
 '아스라함',
 '조용함',
 '복잡함',
 '막강함',
 '예민함',
 '온전함',
 '짤막함',
 '애매모호함',
 '대담함',
 '절묘함',
 '통쾌함',
 '풍성함',
 '명징함',
 '치밀함',
 '따듯함',
 '촉촉함',
 '감미함',
 '잔잔함',
 '팽팽함',
 '청량함',
 '왕성함',
 '긴밀함',
 '극진함',
 '은밀함',
 '민망함',
 '면밀함',
 '미묘함',
 '모호함',
 '후덥지근함',
 '로맨틱함',
 '깜찍함',
 '온순함',
 '껑충함',
 '깐깐함',
 '널널함',
 '여의함',
 '강인함',
 '오묘함',
 '서운함',
 '청초함',
 '온화함',
 '방만함',
 '확연함',
 '정정함',
 '명확함',
 '큼지막함',
 '의연함',
 '잠잠함',
 '순탄함',
 '쾌적함',
 '매콤함',
 '진솔함',
 '영민함',
 '유사함',
 '지루함',
 '희박함',
 '새콤함',
 '영리함',
 '식상함',
 '가뿐함',
 '처연함',
 '간결함',
 '흐뭇함',
 '허술함',
 '여전함',
 '초연함',
 '두근함',
 '확실함',
 '임박함',
 '세련함',
 '절절함',
 '험난함',
 '성급함',
 '널찍함',
 '족함',
 '바람직함',
 '엄정함',
 '창창함',
 '무수함',
 '흥얼함',
 '수려함',
 '진귀함',
 '강함',
 '솔직함',
 '정당함',
 '평이함',
 '희미함',
 '즐비함',
 '홀가분함',
 '죄송함',
 '따뜻함',
 '듬직함',
 '추악함',
 '중함',
 '각별함',
 '묵직함',
 '열악함',
 '덤덤함',
 '심심함',
 '고단함',
 '수더분함',
 '다채함',
 '이러이러

In [357]:
total_keyword_list = total_nva_keyword + total_nxr_keyword
total_keyword_list

['분함',
 '탄탄함',
 '풋풋함',
 '조급함',
 '한결같음',
 '까맣음',
 '길음',
 '틀림없음',
 '선선함',
 '상큼함',
 '싸늘함',
 '잦음',
 '맛남',
 '화창함',
 '벌겋음',
 '조그맣음',
 '쏠쏠함',
 '어리둥절함',
 '별다름',
 '재미있음',
 '눈부심',
 '심심함',
 '세세함',
 '찰짐',
 '단호함',
 '시큰둥함',
 '답답함',
 '어딨음',
 '뚜렷함',
 '웅장함',
 '소소함',
 '쌈',
 '읎음',
 '비릿함',
 '걸맞음',
 '생소함',
 '위대함',
 '납작함',
 '밤늦음',
 '깜찍함',
 '섬세함',
 '난처함',
 '묘함',
 '더딤',
 '구성짐',
 '단정함',
 '어눌함',
 '친함',
 '긴밀함',
 '달달함',
 '터무니없음',
 '미묘함',
 '다급함',
 '동그랗음',
 '새하얗음',
 '굵음',
 '도드라짐',
 '간편함',
 '화려함',
 '나쁨',
 '옹골참',
 '열렬함',
 '공고함',
 '이렇음',
 '푸르름',
 '절실함',
 '어떻음',
 '씀',
 '약함',
 '쓸데없음',
 '따듯함',
 '그러함',
 '불쌍함',
 '발칙함',
 '수두룩함',
 '진정함',
 '귀함',
 '궂음',
 '맛있음',
 '짙푸름',
 '든든함',
 '철없음',
 '능숙함',
 '재밌음',
 '푸름',
 '뒤늦음',
 '가쁨',
 '그럴싸함',
 '섭섭함',
 '느긋함',
 '쎔',
 '혹독함',
 '광활함',
 '정갈함',
 '따스함',
 '완연함',
 '또렷함',
 '좋음',
 '굴곡짐',
 '역함',
 '끔',
 '급격함',
 '간단함',
 '오램',
 '은은함',
 '비림',
 '기막힘',
 '이쁨',
 '밋밋함',
 '만만찮음',
 '수수함',
 '승함',
 '질김',
 '삐뚤음',
 '두드러짐',
 '색다름',
 '바름',
 '흥미진진함',
 '막대함',
 '방정맞음',
 '맑음',
 '크나큼',
 '똑같음',
 '배고픔',
 '거멓음',
 '고름',
 '아낌없음

- 전체 meaning words 리스트

In [362]:
total_nva_meaningwords = adjective_to_noun(total_va_meaningwords)


In [361]:
total_nxr_meaningwords = xr_to_noun(total_xr_meaningwords)

In [360]:
total_meaningwords_list = total_nva_meaningwords + total_nxr_meaningwords
total_meaningwords_list

['분함',
 '어림',
 '거셈',
 '한결같음',
 '드셈',
 '빼어남',
 '길음',
 '힘참',
 '까맣음',
 '멋들어짐',
 '부심',
 '새빨갛음',
 '연함',
 '유함',
 '냉함',
 '편함',
 '뿌옇음',
 '섹시함',
 '벌겋음',
 '환함',
 '조그맣음',
 '진함',
 '얇음',
 '큼',
 '참',
 '짧음',
 '기참',
 '슬픔',
 '둥글음',
 '재미있음',
 '별남',
 '당참',
 '해맑음',
 '눈부심',
 '뜸함',
 '빨갛음',
 '빈틈없음',
 '순함',
 '어질음',
 '변함없음',
 '앙증맞음',
 '독함',
 '후함',
 '여림',
 '멍함',
 '가늘음',
 '무름',
 '아림',
 '쌈',
 '틀림',
 '탁함',
 '많음',
 '강함',
 '싸함',
 '잘생김',
 '힘들음',
 '올바름',
 '정신없음',
 '어리석음',
 '짓궂음',
 '느림',
 '예쁨',
 '세참',
 '우렁참',
 '묘함',
 '속상함',
 '간드러짐',
 '손색없음',
 '장함',
 '서투름',
 '친함',
 '멋있음',
 '벅참',
 '값비쌈',
 '엄청남',
 '찐함',
 '동그랗음',
 '낯설음',
 '새하얗음',
 '굵음',
 '잘음',
 '반함',
 '급함',
 '잘남',
 '도드라짐',
 '바쁨',
 '젊음',
 '밝음',
 '뛰어남',
 '나쁨',
 '앳됨',
 '멋짐',
 '뽀얗음',
 '격함',
 '옹골참',
 '검음',
 '파랗음',
 '푸르름',
 '기운참',
 '굳셈',
 '가녀림',
 '약함',
 '값짐',
 '동떨어짐',
 '다름',
 '쓸데없음',
 '능글맞음',
 '특별남',
 '새까맣음',
 '비쌈',
 '괜찮음',
 '흼',
 '뻔함',
 '끊임없음',
 '귀함',
 '몽글음',
 '궂음',
 '맛있음',
 '희망참',
 '짙푸름',
 '서글픔',
 '짠함',
 '하얗음',
 '철없음',
 '노랗음',
 '재밌음',
 '낡음',
 '활기참',
 '커다랗음',
 '똑바름',
 '어이없음',
 

- 전체 키워드 , meaningwords 리스트 비교

In [363]:
total_keyword_list = set(total_keyword_list)
total_meaningwords_list = set(total_meaningwords_list)

In [364]:
common_keywords = total_keyword_list.intersection(total_meaningwords_list)

print("공통 키워드:", common_keywords)
print('공통 키워드 개수:', len(common_keywords)) #50% 정도 겹침

공통 키워드: {'공고함', '올바름', '별남', '오램', '몽글음', '앙상함', '훌륭함', '싱싱함', '야함', '처연함', '어리석음', '영롱함', '씩씩함', '빨갛음', '애절함', '까칠함', '지나침', '황당함', '강경함', '달콤함', '앙증맞음', '똑바름', '격렬함', '흼', '배부름', '상큼함', '동떨어짐', '다소곳함', '서늘함', '가뿐함', '사근사근함', '배고픔', '깐깐함', '독함', '엄청남', '익숙함', '빈틈없음', '아찔함', '열렬함', '희망참', '빡빡함', '서글픔', '찝찝함', '짠함', '갸름함', '원만함', '비쌈', '동그랗음', '간드러짐', '갑갑함', '든든함', '용감함', '듬직함', '불쾌함', '낯설음', '심심함', '당돌함', '도도함', '관대함', '가늘음', '눈부심', '담백함', '촘촘함', '좋음', '절친함', '진솔함', '쫄깃함', '졸렬함', '한심함', '억울함', '굵음', '싫음', '노골함', '탁함', '소심함', '건실함', '은근함', '영리함', '뜸함', '광활함', '우울함', '새빨갛음', '살벌함', '다름', '쎔', '뻔뻔함', '값비쌈', '단아함', '청량함', '나쁨', '재미있음', '방만함', '과감함', '세세함', '기막힘', '독특함', '평이함', '고상함', '휘황찬란함', '진중함', '쓰라림', '유일함', '낯익음', '틀림', '찐함', '향긋함', '분함', '철없음', '해박함', '후함', '기운참', '똑똑함', '솔깃함', '짓궂음', '화사함', '담대함', '새하얗음', '조그맣음', '약함', '난해함', '강인함', '침울함', '빵빵함', '다정함', '성급함', '아늑함', '뿌옇음', '당당함', '유치함', '날렵함', '느끼함', '힘참', '아픔', '쓸데없음', '큼직함', '느림', '오묘함', '매끈함', '지독함', '면밀함', '매정함', '출중함', '발랄함',

In [365]:
unique_total_keyword_list = total_keyword_list.difference(common_keywords)
unique_meaningwords_list = total_meaningwords_list.difference(common_keywords)

print("기사+블로그 키워드 리스트 :", unique_total_keyword_list) # 200, 300
print("kkm 형태소 키워드 리스트 :", unique_meaningwords_list)


기사+블로그 키워드 리스트 : {'널널함', '특별함', '겸허함', '명징함', '늦음', '읎음', '뭉클함', '볼멤', '영락없음', '단순함', '경미함', '타당함', '온당함', '머쓱함', '팽팽함', '끝없음', '어떡함', '익음', '명확함', '중함', '간략함', '극심함', '방불함', '어떻음', '넓음', '정정함', '적음', '있음', '심각함', '아른함', '수없음', '뚜렷함', '창창함', '희미함', '통통함', '생뚱맞음', '위대함', '떠들썩함', '두근함', '민망함', '완만함', '조그마함', '의미심장함', '진기함', '단출함', '맞이함', '쿵쾅함', '헛됨', '별다름', '참신함', '어중간함', '잠잠함', '얕음', '멸함', '막중함', '간소함', '자잘함', '큼지막함', '난처함', '은은함', '교묘함', '뻑뻑함', '개운함', '느슨함', '크나큼', '적절함', '아슬함', '쾌활함', '불구함', '간단함', '무관함', '극진함', '시끌함', '공공연함', '좋아함', '한가함', '열악함', '완연함', '술렁함', '비림', '달짝지근함', '쾌적함', '각짐', '부적절함', '족함', '비롯함', '핫함', '설음', '유사함', '심오함', '평평함', '달달함', '틀림없음', '또렷함', '씁쓸함', '잦음', '고소함', '넉넉함', '가지런함', '폭신함', '임박함', '복잡함', '시급함', '본격함', '밤늦음', '똑같음', '쫀득함', '지난함', '떨떠름함', '자명함', '명쾌함', '고름', '비릿함', '따끈함', '드물음', '한산함', '적잖음', '마지못함', '정교함', '쫑알함', '긴가민가함', '의아함', '묽음', '면함', '금', '죄송함', '유창함', '무난함', '아득함', '껑충함', '희귀함', '너름', '구성짐', '광범함', '자세함', '성함', '무수함', '짜릿함', '기나길음', '짙음', '어마어마함',

In [366]:
# 전체 키워드, meaning words 키워드 개수

len(total_keyword_list), len(total_meaningwords_list)

(840, 462)

In [367]:
# 공통 키워드 개수, 전체-공통, meaningwords-공통
len(common_keywords), len(unique_total_keyword_list), len(unique_meaningwords_list)

(462, 378, 0)

In [368]:
unique_total_keyword_list

{'가능함',
 '가쁨',
 '가지런함',
 '가파름',
 '각짐',
 '간결함',
 '간단함',
 '간략함',
 '간소함',
 '간편함',
 '개운함',
 '거창함',
 '걸맞음',
 '견고함',
 '겸허함',
 '경미함',
 '고고함',
 '고름',
 '고소함',
 '곡진함',
 '공공연함',
 '공교함',
 '광범함',
 '괜함',
 '굉장함',
 '교묘함',
 '구성짐',
 '굴곡짐',
 '굵직함',
 '귀중함',
 '그러함',
 '그럴싸함',
 '그렇음',
 '그윽함',
 '극명함',
 '극심함',
 '극진함',
 '금',
 '급격함',
 '기괴함',
 '기나길음',
 '기발함',
 '기이함',
 '긴가민가함',
 '긴밀함',
 '깊숙함',
 '깊음',
 '까마득함',
 '껑충함',
 '끔',
 '끝없음',
 '나른함',
 '난감함',
 '난데없음',
 '난처함',
 '낮음',
 '너름',
 '넉넉함',
 '널널함',
 '널찍함',
 '넓음',
 '높아짐',
 '높음',
 '누렇음',
 '느슨함',
 '능숙함',
 '늦음',
 '다급함',
 '다름없음',
 '다분함',
 '다채함',
 '단순함',
 '단조함',
 '단출함',
 '단호함',
 '달달함',
 '달음',
 '달짝지근함',
 '당연함',
 '대견함',
 '더딤',
 '돈독함',
 '동일함',
 '두근함',
 '두리번함',
 '뒤늦음',
 '뒤숭숭함',
 '드넓음',
 '드물음',
 '따끈함',
 '따끔함',
 '딱딱함',
 '떠들석함',
 '떠들썩함',
 '떨떠름함',
 '또렷함',
 '똑같음',
 '뚜렷함',
 '뜨끈함',
 '뜻깊음',
 '마다함',
 '마땅함',
 '마지못함',
 '막대함',
 '막론함',
 '막막함',
 '막연함',
 '막중함',
 '말랑함',
 '맛남',
 '맞이함',
 '매콤함',
 '맹렬함',
 '머나멀음',
 '머쓱함',
 '머지않음',
 '멀음',
 '멀쩡함',
 '멋쩍음',
 '면함',
 '멸함',
 '명백함',
 '명징함',
 '명쾌함',
 '명확함',
 

In [370]:
total_unique_xr_keyword

{'가능',
 '가지런',
 '간결',
 '간단',
 '간략',
 '간소',
 '간편',
 '강',
 '개운',
 '거창',
 '견고',
 '겸허',
 '경미',
 '고소',
 '곡진',
 '공공연',
 '공교',
 '광범',
 '괜',
 '굉장',
 '교묘',
 '귀중',
 '그러',
 '그윽',
 '극명',
 '극진',
 '급격',
 '기괴',
 '기발',
 '기이',
 '긴가민가',
 '긴밀',
 '깊숙',
 '까마득',
 '껑충',
 '나른',
 '난감',
 '넉넉',
 '널널',
 '널찍',
 '능숙',
 '다분',
 '다채',
 '단순',
 '단조',
 '단출',
 '달짝지근',
 '당연',
 '대견',
 '돈독',
 '동일',
 '두근',
 '두리번',
 '뒤숭숭',
 '따끔',
 '딱딱',
 '떠들석',
 '떠들썩',
 '떨떠름',
 '또렷',
 '뚜렷',
 '뜸',
 '마다',
 '마땅',
 '막대',
 '막론',
 '막막',
 '막연',
 '막중',
 '말랑',
 '맞이',
 '매콤',
 '맹렬',
 '머쓱',
 '멀쩡',
 '면',
 '멸',
 '명백',
 '명징',
 '명쾌',
 '명확',
 '모호',
 '몽롱',
 '무관',
 '무근',
 '무난',
 '무도',
 '무방',
 '무색',
 '무성',
 '무수',
 '뭉근',
 '뭉클',
 '미묘',
 '미미',
 '미세',
 '미약',
 '민망',
 '방대',
 '방불',
 '복잡',
 '복잡다단',
 '본격',
 '분분',
 '분주',
 '불가피',
 '불구',
 '불사',
 '불순',
 '비등',
 '비롯',
 '비슷',
 '비슷비슷',
 '비아냥',
 '빈번',
 '빠듯',
 '뿌듯',
 '사소',
 '상세',
 '상이',
 '생경',
 '서운',
 '선명',
 '성대',
 '세밀',
 '수더분',
 '수두룩',
 '수월',
 '순탄',
 '술렁',
 '시급',
 '시끌',
 '신기',
 '심',
 '심각',
 '심상',
 '심오',
 '쌉쌀',
 '쏠쏠',
 '씁쓸',
 '아른',

In [1]:
text_unique_xr_keywords = ['가지런',
 '거창',
 '견고',
 '겸허',
 '굉장',
 '교묘',
 '귀중',
 '그윽',
 '기괴',
 '기발',
 '기이',
 '나른',
 '난감',
 '넉넉',
 '널널',
 '능숙',
 '다채',
 '단순',
 '단조',
 '단출',
 '달짝지근',
 '대견',
 '돈독',
 '두근',
 '딱딱',
 '떠들석',
 '떠들썩',
 '떨떠름',
 '또렷',
 '뚜렷',
 '막막',
 '막연',
 '말랑',
 '매콤',
 '맹렬',
 '머쓱',
 '멀쩡',
 '명쾌',
 '명확',
 '모호',
 '몽롱',
 '무난',
 '무도',
 '무색',
 '뭉근',
 '뭉클',
 '미묘',
 '미미',
 '미세',
 '미약',
 '복잡',
 '복잡다단',
 '분주',
 '불순',
 '비아냥',
 '사소',
 '상세',
 '생경',
 '서운',
 '선명',
 '성대',
 '세밀',
 '수더분',
 '수월',
 '순탄',
 '술렁',
 '시끌',
 '신기',
 '심각',
 '심상',
 '심오',
 '쌉쌀',
 '쏠쏠',
 '씁쓸',
 '아스라',
 '어마어마',
 '어중간',
 '어지간',
 '엄',
 '엄정',
 '열악',
 '온전',
 '완만',
 '용이',
 '위대',
 '유창',
 '으르렁',
 '은은',
 '의미심장',
 '의아',
 '이룩',
 '자세',
 '잠잠',
 '잡다',
 '적당',
 '적절',
 '절묘',
 '정교',
 '정당',
 '정연',
 '정정',
 '정확',
 '조그마',
 '주요',
 '준수',
 '중요',
 '즐비',
 '지대',
 '진기',
 '짜릿',
 '짤막',
 '찬란',
 '참신',
 '창창',
 '청청',
 '촉촉',
 '축축',
 '충분',
 '치밀',
 '친밀',
 '칼칼',
 '쾌적',
 '쿵쾅',
 '큼지막',
 '탁월',
 '텁텁',
 '테이크',
 '톡톡',
 '통통',
 '특별',
 '특유',
 '특이',
 '팔랑',
 '팽팽',
 '편',
 '평평',
 '폭신',
 '풍부',
 '풍성',
 '한가',
 '합당',
 '험난',
 '혹독',
 '홀가분',
 '화',
 '화기애애',
 '확고',
 '확실',
 '확연',
 '환',
 '획일',
 '후덥지근',
 '흐릿',
 '흐뭇',
 '흔',
 '희귀',
 '희미',
 '희박']

In [2]:
text_unique_va_keywords = [
 '간절하',
 '강렬하',
 '경쾌하',
 '고고하',
 '과감하',
 '과하',
 '관대하',
 '굉장하',
 '교묘하',
 '구수하',
 '굳건하',
 '굵직하',
 '깐깐하',
 '깜찍하',
 '깨끗하',
 '꾸준하',
 '끈끈하',
 '나른하',
 '난감하',
 '난처하',
 '난해하',
 '날렵하',
 '날씬하',
 '납작하',
 '누렇',
 '느긋하',
 '느끼하',
 '능수능란하',
 '능숙하',
 '단단하',
 '단정하',
 '단호하',
 '달달하',
 '달콤하',
 '담담하',
 '담백하',
 '답답하',
 '대견하',
 '대단하',
 '든든하',
 '따듯하',
 '따뜻하',
 '따스하',
 '딱딱하',
 '또렷하',
 '뚜렷하',
 '뜨끈하',
 '뜻깊',
 '막중하',
 '매끈하',
 '매콤하',
 '무난하',
 '미미하',
 '미약하',
 '밋밋하',
 '발랄하',
 '발칙하',
 '복잡하',
 '불쌍하',
 '뻔뻔하',
 '뿌듯하',
 '상큼하',
 '새콤하',
 '생뚱맞',
 '생생하',
 '생소하',
 '서운하',
 '섬세하',
 '섭섭하',
 '세심하',
 '소소하',
 '수수하',
 '시원하',
 '신기하',
 '신선하',
 '싸늘하',
 '씁쓸하',
 '씩씩하',
 '애절하',
 '어눌하',
 '어색하',
 '어설프',
 '얼떨떨하',
 '열악하',
 '예민하',
 '오지',
 '우아하',
 '웅장하',
 '위대하',
 '유연하',
 '유창하',
 '잔잔하',
 '쟁쟁하',
 '저렴하',
 '절실하',
 '절친하',
 '정갈하',
 '조그마하',
 '조급하',
 '지루하',
 '지저분하',
 '진솔하',
 '진지하',
 '차분하',
 '참신하',
 '촉촉하',
 '친근하',
 '쾌활하',
 '쿨하',
 '큼직하',
 '탁월하',
 '탄탄하',
 '투박하',
 '튼튼하',
 '평범하',
 '풋풋하',
 '핫하',
 '향긋하',
 '화기애애하',
 '화려하',
 '화사하',
 '화창하',
 '활발하',
 '훈훈하',
 '훌륭하',]

In [369]:
total_unique_va_keyword

{'가쁘',
 '가파르',
 '각지',
 '간단하',
 '간략하',
 '간절하',
 '간편하',
 '강렬하',
 '걸맞',
 '견고하',
 '경쾌하',
 '고고하',
 '고르',
 '고소하',
 '공고하',
 '공공연하',
 '과감하',
 '과하',
 '관대하',
 '광활하',
 '괜하',
 '굉장하',
 '교묘하',
 '구성지',
 '구수하',
 '굳건하',
 '굴곡지',
 '굵직하',
 '궁금하',
 '그',
 '그러하',
 '그럴싸하',
 '그렇',
 '극심하',
 '근사하',
 '근소하',
 '급격하',
 '기괴하',
 '기나길',
 '기발하',
 '긴밀하',
 '깊',
 '깐깐하',
 '깜찍하',
 '깨끗하',
 '꾸준하',
 '끄',
 '끈끈하',
 '끝없',
 '나른하',
 '난감하',
 '난데없',
 '난처하',
 '난해하',
 '날렵하',
 '날씬하',
 '납작하',
 '낮',
 '너르',
 '넉넉하',
 '넓',
 '높',
 '높아지',
 '누렇',
 '느긋하',
 '느끼하',
 '느슨하',
 '능수능란하',
 '능숙하',
 '늦',
 '다급하',
 '다름없',
 '단단하',
 '단정하',
 '단호하',
 '달',
 '달달하',
 '달콤하',
 '담담하',
 '담백하',
 '답답하',
 '대견하',
 '대단하',
 '더디',
 '도도하',
 '뒤늦',
 '드넓',
 '드물',
 '든든하',
 '따끈하',
 '따듯하',
 '따뜻하',
 '따스하',
 '딱딱하',
 '또렷하',
 '똑같',
 '뚜렷하',
 '뜨끈하',
 '뜻깊',
 '마땅하',
 '마지못하',
 '막대하',
 '막중하',
 '맛나',
 '매끈하',
 '매콤하',
 '머나멀',
 '머지않',
 '멀',
 '멀쩡하',
 '멋쩍',
 '명확하',
 '목마르',
 '못지않',
 '무관하',
 '무난하',
 '무색하',
 '무성하',
 '묵직하',
 '문제없',
 '묽',
 '미묘하',
 '미미하',
 '미약하',
 '밋밋하',
 '바람직하',
 '발랄하',
 '발칙하',
 '밤늦',
 

In [1]:
output_list = [f"'{word.replace(',', '').strip()}'," for word in xr_meaningwords]
output_text = " ".join(output_list)
print(output_text)

NameError: ignored